In [2]:
import pandas as pd
import re
import datetime
from datetime import *

In [2]:
pd.set_option('display.max_colwidth',-1)

In [3]:
pd.set_option('display.max_rows', 500)
pd.options.mode.chained_assignment = None 

In [ ]:
# Importing or Exporting from excel can take only a max of 32767 characters per cell. Though Excel sheet from Craig contains max of 32767 chars, when imported into excel spaces and line breaks are created as characters and hence entire data is not imported and it is truncated
# Issue can be solved by directly importing from database etc

In [ ]:
# # Install Packages
# # https://github.com/akoumjian/datefinder
# import sys 
# !{sys.executable} -m pip install datefinder

# import sys 
# !{sys.executable} -m pip install num2words 

In [ ]:
### DataSets
# ClinicalNotes - Obtained after initial pre-processing
# ClinicalNotes_error - Observations at which DateTime function has errors
# ClinicalNotes_NoDateErrors - Obtained after DateTime format function (If errors are obtained while processing the datatime function, those obs have been removed)
# ClinicalNotes_UserEntered - subset of ClinicalNotes_NoDateErrors where source type = User entered (data is sorted to remove duplicates)
# ClinicalNotes_UserEntered_Explode - ClinicalNotes_UserEntered exploded based on line breaks
# ClinicalNotes_UserEntered_Explode_Num - ClinicalNotes_UserEntered_Explode that contains Numeric values


In [10]:
# Row = 963 - There is some error in ChangeDateTimeFormatx function
# Row = 984 - 4/6 is written as 4\6 so DateTime function wont work
# Row = 385,9134 - Typo... \ is entered instead of fullstop
# Escape char '\' should not be removed before exploding the data as escape char are part of line breaks


# textToSearch = str(ClinicalNotes[ClinicalNotes.Row==984].Text)
# textToSearch = "last admit 2/28 s/p aspiration pna given vanc and cefepime.  discharged from here 1/7 severe sepsis"
textToSearch = "\n12/26 - hx of multiple admissions for respiratory distress associated with copd and pa pneumonia. pt is being currently treated with zosyn "
ChangeDateTimeFormat(textToSearch)

'\nDecember26 - hx of multiple admissions for respiratory distress associated with copd and pa pneumonia. pt is being currently treated with zosyn '

In [3]:
import pickle
ClinicalNotes = pd.read_pickle("Py DataFrames/ClinicalNotes.pkl") # Obtained after initial pre-processing
ClinicalNotes_NoDateErrors = pd.read_pickle("Py DataFrames/ClinicalNotes_NoDateErrors.pkl") # Obtained after DateTime format function
# ClinicalNotes_UserEntered = pd.read_pickle("Py DataFrames/ClinicalNotes_UserEntered.pkl") # SourceType=UserEntered (subset of ClinicalNotes_NoDateErrors)

In [4]:
UserEntered_Num = pd.read_pickle("Py DataFrames/UserEntered_Num.pkl") # Obtained after initial pre-processing

In [ ]:
ClinicalNotes = pd.read_excel('Craig_2019Aug7th.xlsx')

# Strip white space
# ClinicalNotes["Text"] = ClinicalNotes["Text"].str.strip()

# convert text to lowercase
ClinicalNotes["Text"] = ClinicalNotes["Text"].str.lower()

# Convert index to column
ClinicalNotes['Row'] = ClinicalNotes.index

# Remove records with source type = ADT, Lab & Order
ClinicalNotes = ClinicalNotes[ClinicalNotes.SourceType.isin(["User Entered","Documentation"])]

# Remove records based on number of words and if there is a number present or not and based on length of sentence
# For eg: Bmi>40 is a single word has a number but length is less
# This example has number of words=1 but length of sentence is long "wprnbwygkmfydj3ijapkf7y15yopqc0lgey2a0iryfkxbvs1zeyjm5c\r\n0vt1iwgfwijef62vbd2j+9ofnokh+moppxxdv7tzunjjcr6/peo0fticiqsb4fli7wudgfonhzab\r\nixs3biz"

# Number of words in a sentence
# df['totalwords'] = df['col'].str.split().str.len() #if /r or /n is present it is considered as space 
# df['totalwords'] = df['col'].str.count(' ') + 1 #if space is present or not
# df['totalwords'] = [len(x.split()) for x in df['col'].tolist()] #
ClinicalNotes['TotalWords'] = ClinicalNotes['Text'].str.count(' ') + 1

# Find the length of the sentence
ClinicalNotes['TextLength'] = ClinicalNotes['Text'].str.len() 

# Check if the Text has a Numeric in it or not
ClinicalNotes['NumericInText'] = ClinicalNotes['Text'].str.contains(("\d"), regex=True)

# Sort data
ClinicalNotes = ClinicalNotes.sort_values(by=['TotalWords','TextLength','NumericInText'], ascending=[True, True, True])

# Remove rows based on the conditions
Removed_ClinicalNotes = ClinicalNotes[(((ClinicalNotes.TotalWords<=5) & (ClinicalNotes.TextLength>500)) | (ClinicalNotes.NumericInText==False))]

## Remove 
# TotalWords<=3 & TextLength>300
# NumericInText is not present
ClinicalNotes = ClinicalNotes[~(((ClinicalNotes.TotalWords<=3) & (ClinicalNotes.TextLength>500)) | (ClinicalNotes.NumericInText==False))]

# Removes special characters from the beginning of a string
import re
ClinicalNotes['Text'] = ClinicalNotes['Text'].str.replace(r"^\W+", "")

# Replace quotes in Text
ClinicalNotes['Text'] = ClinicalNotes['Text'].str.replace('"', "")

# Export data
ClinicalNotes.to_excel('Py DataFrames/ClinicalNotes.xlsx',index=False)
ClinicalNotes.to_pickle("Py DataFrames/ClinicalNotes.pkl")

In [88]:
# Export data
ClinicalNotes.to_excel('Py DataFrames/ClinicalNotes.xlsx',index=False)
ClinicalNotes.to_pickle("Py DataFrames/ClinicalNotes.pkl")

In [6]:
ClinicalNotes.tail()

,Date,Source,SourceType,Text,Row,TotalWords,TextLength,NumericInText
22218,2019-08-06 14:30:25,Patient Order Documentation,Documentation,coordination of care initial assessment entere...,22218,5355,32767,True
34722,2019-08-07 09:52:20,Patient Order Documentation,Documentation,"patient education instructions\nname: judkins,...",34722,5570,32767,True
21132,2019-08-06 13:18:41,Patient Order Documentation,Documentation,"daniels, richard n \n dob: 11/04/1953 \n mrn: ...",21132,5614,32767,True
21936,2019-08-06 14:13:16,Patient Order Documentation,Documentation,"daniels, richard n \n dob: 11/04/1953 \n mrn: ...",21936,5617,32767,True
34815,2019-08-07 09:54:39,Patient Order Documentation,Documentation,"judkins, cloieesboy \n dob: 08/05/2019 \n mrn:...",34815,6106,32767,True


In [ ]:
# Identifies only single date present
# Check_head['Date'] = Check_head['Text'].str.extract(r'(\d+/\d+)', expand=True)

# Identifies only if 2 dates are present
# Check_head['Date'] = Check_head.Text.str.extract(r'(\d+/\d+)(\D+)(\d+/\d+)', expand=True)

In [ ]:
# # Find dates in m/d/y format
# Check_head['Month_Date_Year'] = Check_head["Text"].str.findall(r'\d+/\d+/\d+')

# # Find dates in m/d format
# Check_head['Month_date'] = Check_head["Text"].str.findall(r'\d+/\d+')

In [ ]:
# Row - 28025
#  wbc 
# 13.6 k/mcl (high) 
# 08/06/2019 16:22 cdt 
    ---So data needs to be exploded only after dates conversion 
# if Text is in such a way that "Any text "\n" Value" then replace \n with space --- This needs to be done after removing dates

In [ ]:
# 	systolic blood pressure	101 mm
# systolic blood pressure\t101 mmhg\t\n\t

# '\t' needs to be replaced with " " space

In [ ]:
# import datefinder
# matches = datefinder.find_dates(textToSearch)
# for match in matches:
#     print(match)

# # 24889 - [july 15, august 2004, july 2009, november 2007, june 2012, january 2018] 
# # 21572    [july 18, august 3, august 6, august 4, august 3, august 3]
# # 30687    [july 21, february 2019]  
# MONTHS_PATTERN = r"january\d+|february\d+|march\d+|april\d+|may\d+|june\d+|july\d+|august\d+|september\d+|october\d+|november\d+|december\d+|january \d+|february \d+|march \d+|april \d+|may \d+|june \d+|july \d+|august \d+|september \d+|october \d+|november \d+|december \d+|jan\d+|jan \d+|feb\d+|feb \d+|mar\d+|mar \d+|apr\d+|apr \d+|may\d+|may \d+|jun\d+|jun \d+|jul\d+|jul \d+|aug\d+|aug \d+|sep\d+|sept \d+|oct\d+|oct \d+|nov\d+|nov \d+|dec\d+|dec \d+"

# ClinicalNotes["Text"].str.findall(MONTHS_PATTERN)

#### Find the observation where ChangeDateTimeFormat function throws error

In [6]:
# Pass
def ErrorINChangeDateTimeFormat(TextwithDates):
    try:
        # Replace sept with sep            
        for match in re.findall("sept \d+", TextwithDates):
            match_rep = re.sub('sept','sep' ,match)
            TextwithDates = re.sub(match, match_rep, TextwithDates)
            
        # Convert jan & january independently     
        suffix = r"st|nd|rd|th"
    
        # Extract Month_Year
        Month_Year = "january \d{4}|february \d{4}|march \d{4}|april \d{4}|may \d{4}|june \d{4}|july \d{4}|august \d{4}|september \d{4}|october \d{4}|november \d{4}|december \d{4}"
        for match in re.findall(Month_Year, TextwithDates):
            datetime_object = datetime.strptime(match,"%B %Y")
            dateNewFormat = datetime_object.strftime("%Y%B")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
        # Mon_Year
        Mon_Year = "jan \d{4}|feb \d{4}|mar \d{4}|apr \d{4}|may \d{4}|jun \d{4}|jul \d{4}|aug \d{4}|sep \d{4}|oct \d{4}|nov \d{4}|dec \d{4}"
        for match in re.findall(Mon_Year, TextwithDates):
            datetime_object = datetime.strptime(match,"%b %Y")
            dateNewFormat = datetime_object.strftime("%Y%B")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        # No Space between month and date
        MonDD_HMS = r"jan\d+st \d+:\d+:\d+|feb\d+st \d+:\d+:\d+|mar\d+st \d+:\d+:\d+|apr\d+st \d+:\d+:\d+|may\d+st \d+:\d+:\d+|jun\d+st \d+:\d+:\d+|jul\d+st \d+:\d+:\d+|aug\d+st \d+:\d+:\d+|sep\d+st \d+:\d+:\d+|oct\d+st \d+:\d+:\d+|nov\d+st \d+:\d+:\d+|dec\d+st \d+:\d+:\d+ \
           jan\d+nd \d+:\d+:\d+|feb\d+nd \d+:\d+:\d+|mar\d+nd \d+:\d+:\d+|apr\d+nd \d+:\d+:\d+|may\d+nd \d+:\d+:\d+|jun\d+nd \d+:\d+:\d+|jul\d+nd \d+:\d+:\d+|aug\d+nd \d+:\d+:\d+|sep\d+nd \d+:\d+:\d+|oct\d+nd \d+:\d+:\d+|nov\d+nd \d+:\d+:\d+|dec\d+nd \d+:\d+:\d+ \
           jan\d+rd \d+:\d+:\d+|feb\d+rd \d+:\d+:\d+|mar\d+rd \d+:\d+:\d+|apr\d+rd \d+:\d+:\d+|may\d+rd \d+:\d+:\d+|jun\d+rd \d+:\d+:\d+|jul\d+rd \d+:\d+:\d+|aug\d+rd \d+:\d+:\d+|sep\d+rd \d+:\d+:\d+|oct\d+rd \d+:\d+:\d+|nov\d+rd \d+:\d+:\d+|dec\d+rd \d+:\d+:\d+ \
           jan\d+th \d+:\d+:\d+|feb\d+th \d+:\d+:\d+|mar\d+th \d+:\d+:\d+|apr\d+th \d+:\d+:\d+|may\d+th \d+:\d+:\d+|jun\d+th \d+:\d+:\d+|jul\d+th \d+:\d+:\d+|aug\d+th \d+:\d+:\d+|sep\d+th \d+:\d+:\d+|oct\d+th \d+:\d+:\d+|nov\d+th \d+:\d+:\d+|dec\d+th \d+:\d+:\d+ \
           jan\d+ \d+:\d+:\d+|feb\d+ \d+:\d+:\d+|mar\d+ \d+:\d+:\d+|apr\d+ \d+:\d+:\d+|may\d+ \d+:\d+:\d+|jun\d+ \d+:\d+:\d+|jul\d+ \d+:\d+:\d+|aug\d+ \d+:\d+:\d+|sep\d+ \d+:\d+:\d+|oct\d+ \d+:\d+:\d+|nov\d+ \d+:\d+:\d+|dec\d+ \d+:\d+:\d+"
        for match in re.findall(MonDD_HMS, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            datetime_object = datetime.strptime(match_rep,"%b%d %H:%M:%S")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
        MonDD_HM = r"jan\d+st \d+:\d+|feb\d+st \d+:\d+|mar\d+st \d+:\d+|apr\d+st \d+:\d+|may\d+st \d+:\d+|jun\d+st \d+:\d+|jul\d+st \d+:\d+|aug\d+st \d+:\d+|sep\d+st \d+:\d+|oct\d+st \d+:\d+|nov\d+st \d+:\d+|dec\d+st \d+:\d+ \
           jan\d+nd \d+:\d+|feb\d+nd \d+:\d+|mar\d+nd \d+:\d+|apr\d+nd \d+:\d+|may\d+nd \d+:\d+|jun\d+nd \d+:\d+|jul\d+nd \d+:\d+|aug\d+nd \d+:\d+|sep\d+nd \d+:\d+|oct\d+nd \d+:\d+|nov\d+nd \d+:\d+|dec\d+nd \d+:\d+ \
           jan\d+rd \d+:\d+|feb\d+rd \d+:\d+|mar\d+rd \d+:\d+|apr\d+rd \d+:\d+|may\d+rd \d+:\d+|jun\d+rd \d+:\d+|jul\d+rd \d+:\d+|aug\d+rd \d+:\d+|sep\d+rd \d+:\d+|oct\d+rd \d+:\d+|nov\d+rd \d+:\d+|dec\d+rd \d+:\d+ \
           jan\d+th \d+:\d+|feb\d+th \d+:\d+|mar\d+th \d+:\d+|apr\d+th \d+:\d+|may\d+th \d+:\d+|jun\d+th \d+:\d+|jul\d+th \d+:\d+|aug\d+th \d+:\d+|sep\d+th \d+:\d+|oct\d+th \d+:\d+|nov\d+th \d+:\d+|dec\d+th \d+:\d+ \
           jan\d+ \d+:\d+|feb\d+ \d+:\d+|mar\d+ \d+:\d+|apr\d+ \d+:\d+|may\d+ \d+:\d+|jun\d+ \d+:\d+|jul\d+ \d+:\d+|aug\d+ \d+:\d+|sep\d+ \d+:\d+|oct\d+ \d+:\d+|nov\d+ \d+:\d+|dec\d+ \d+:\d+"
           
        for match in re.findall(MonDD_HM, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            datetime_object = datetime.strptime(match_rep,"%b%d %H:%M")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
        MonDD = r"jan\d+st|feb\d+st|mar\d+st|apr\d+st|may\d+st|jun\d+st|jul\d+st|aug\d+st|sep\d+st|oct\d+st|nov\d+st|dec\d+st \
           jan\d+nd|feb\d+nd|mar\d+nd|apr\d+nd|may\d+nd|jun\d+nd|jul\d+nd|aug\d+nd|sep\d+nd|oct\d+nd|nov\d+nd|dec\d+nd \
           jan\d+rd|feb\d+rd|mar\d+rd|apr\d+rd|may\d+rd|jun\d+rd|jul\d+rd|aug\d+rd|sep\d+rd|oct\d+rd|nov\d+rd|dec\d+rd \
           jan\d+th|feb\d+th|mar\d+th|apr\d+th|may\d+th|jun\d+th|jul\d+th|aug\d+th|sep\d+th|oct\d+th|nov\d+th|dec\d+th \
           jan\d+|feb\d+|mar\d+|apr\d+|may\d+|jun\d+|jul\d+|aug\d+|sep\d+|oct\d+|nov\d+|dec\d+"
        for match in re.findall(MonDD, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            datetime_object = datetime.strptime(match_rep,"%b%d")
            dateNewFormat = datetime_object.strftime("%B%#d")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        # Space between month and date
        Mon_DD_HMS = r"jan \d+st \d+:\d+:\d+|feb \d+st \d+:\d+:\d+|mar \d+st \d+:\d+:\d+|apr \d+st \d+:\d+:\d+|may \d+st \d+:\d+:\d+|jun \d+st \d+:\d+:\d+|jul \d+st \d+:\d+:\d+|aug \d+st \d+:\d+:\d+|sep \d+st \d+:\d+:\d+|oct \d+st \d+:\d+:\d+|nov \d+st \d+:\d+:\d+|dec \d+st \d+:\d+:\d+ \
           jan \d+nd \d+:\d+:\d+|feb \d+nd \d+:\d+:\d+|mar \d+nd \d+:\d+:\d+|apr \d+nd \d+:\d+:\d+|may \d+nd \d+:\d+:\d+|jun \d+nd \d+:\d+:\d+|jul \d+nd \d+:\d+:\d+|aug \d+nd \d+:\d+:\d+|sep \d+nd \d+:\d+:\d+|oct \d+nd \d+:\d+:\d+|nov \d+nd \d+:\d+:\d+|dec \d+nd \d+:\d+:\d+ \
           jan \d+rd \d+:\d+:\d+|feb \d+rd \d+:\d+:\d+|mar \d+rd \d+:\d+:\d+|apr \d+rd \d+:\d+:\d+|may \d+rd \d+:\d+:\d+|jun \d+rd \d+:\d+:\d+|jul \d+rd \d+:\d+:\d+|aug \d+rd \d+:\d+:\d+|sep \d+rd \d+:\d+:\d+|oct \d+rd \d+:\d+:\d+|nov \d+rd \d+:\d+:\d+|dec \d+rd \d+:\d+:\d+ \
           jan \d+th \d+:\d+:\d+|feb \d+th \d+:\d+:\d+|mar \d+th \d+:\d+:\d+|apr \d+th \d+:\d+:\d+|may \d+th \d+:\d+:\d+|jun \d+th \d+:\d+:\d+|jul \d+th \d+:\d+:\d+|aug \d+th \d+:\d+:\d+|sep \d+th \d+:\d+:\d+|oct \d+th \d+:\d+:\d+|nov \d+th \d+:\d+:\d+|dec \d+th \d+:\d+:\d+ \
           jan \d+ \d+:\d+:\d+|feb \d+ \d+:\d+:\d+|mar \d+ \d+:\d+:\d+|apr \d+ \d+:\d+:\d+|may \d+ \d+:\d+:\d+|jun \d+ \d+:\d+:\d+|jul \d+ \d+:\d+:\d+|aug \d+ \d+:\d+:\d+|sep \d+ \d+:\d+:\d+|oct \d+ \d+:\d+:\d+|nov \d+ \d+:\d+:\d+|dec \d+ \d+:\d+:\d+"
        for match in re.findall(Mon_DD_HMS, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            datetime_object = datetime.strptime(match_rep,"%b %d %H:%M:%S")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
        Mon_DD_HM = r"jan \d+st \d+:\d+|feb \d+st \d+:\d+|mar \d+st \d+:\d+|apr \d+st \d+:\d+|may \d+st \d+:\d+|jun \d+st \d+:\d+|jul \d+st \d+:\d+|aug \d+st \d+:\d+|sep \d+st \d+:\d+|oct \d+st \d+:\d+|nov \d+st \d+:\d+|dec \d+st \d+:\d+ \
           jan \d+nd \d+:\d+|feb \d+nd \d+:\d+|mar \d+nd \d+:\d+|apr \d+nd \d+:\d+|may \d+nd \d+:\d+|jun \d+nd \d+:\d+|jul \d+nd \d+:\d+|aug \d+nd \d+:\d+|sep \d+nd \d+:\d+|oct \d+nd \d+:\d+|nov \d+nd \d+:\d+|dec \d+nd \d+:\d+ \
           jan \d+rd \d+:\d+|feb \d+rd \d+:\d+|mar \d+rd \d+:\d+|apr \d+rd \d+:\d+|may \d+rd \d+:\d+|jun \d+rd \d+:\d+|jul \d+rd \d+:\d+|aug \d+rd \d+:\d+|sep \d+rd \d+:\d+|oct \d+rd \d+:\d+|nov \d+rd \d+:\d+|dec \d+rd \d+:\d+ \
           jan \d+th \d+:\d+|feb \d+th \d+:\d+|mar \d+th \d+:\d+|apr \d+th \d+:\d+|may \d+th \d+:\d+|jun \d+th \d+:\d+|jul \d+th \d+:\d+|aug \d+th \d+:\d+|sep \d+th \d+:\d+|oct \d+th \d+:\d+|nov \d+th \d+:\d+|dec \d+th \d+:\d+ \
           jan \d+ \d+:\d+|feb \d+ \d+:\d+|mar \d+ \d+:\d+|apr \d+ \d+:\d+|may \d+ \d+:\d+|jun \d+ \d+:\d+|jul \d+ \d+:\d+|aug \d+ \d+:\d+|sep \d+ \d+:\d+|oct \d+ \d+:\d+|nov \d+ \d+:\d+|dec \d+ \d+:\d+"
        for match in re.findall(Mon_DD_HM, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            datetime_object = datetime.strptime(match_rep,"%b %d %H:%M")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        Mon_DD = r"jan \d+st|feb \d+st|mar \d+st|apr \d+st|may \d+st|jun \d+st|jul \d+st|aug \d+st|sep \d+st|oct \d+st|nov \d+st|dec \d+st \
           jan \d+nd|feb \d+nd|mar \d+nd|apr \d+nd|may \d+nd|jun \d+nd|jul \d+nd|aug \d+nd|sep \d+nd|oct \d+nd|nov \d+nd|dec \d+nd \
           jan \d+rd|feb \d+rd|mar \d+rd|apr \d+rd|may \d+rd|jun \d+rd|jul \d+rd|aug \d+rd|sep \d+rd|oct \d+rd|nov \d+rd|dec \d+rd \
           jan \d+th|feb \d+th|mar \d+th|apr \d+th|may \d+th|jun \d+th|jul \d+th|aug \d+th|sep \d+th|oct \d+th|nov \d+th|dec \d+th \
           jan \d+|feb \d+|mar \d+|apr \d+|may \d+|jun \d+|jul \d+|aug \d+|sep \d+|oct \d+|nov \d+|dec \d+"
        for match in re.findall(Mon_DD, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            datetime_object = datetime.strptime(match_rep,"%b %d")
            dateNewFormat = datetime_object.strftime("%B%#d")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        # Space between month and date
        Month_DD_HMS = r"january \d+st \d+:\d+:\d+|february \d+st \d+:\d+:\d+|march \d+st \d+:\d+:\d+|april \d+st \d+:\d+:\d+|may \d+st \d+:\d+:\d+|june \d+st \d+:\d+:\d+|july \d+st \d+:\d+:\d+|august \d+st \d+:\d+:\d+|september \d+st \d+:\d+:\d+|october \d+st \d+:\d+:\d+|november \d+st \d+:\d+:\d+|december \d+st \d+:\d+:\d+ \
           january \d+nd \d+:\d+:\d+|february \d+nd \d+:\d+:\d+|march \d+nd \d+:\d+:\d+|april \d+nd \d+:\d+:\d+|may \d+nd \d+:\d+:\d+|june \d+nd \d+:\d+:\d+|july \d+nd \d+:\d+:\d+|august \d+nd \d+:\d+:\d+|september \d+nd \d+:\d+:\d+|october \d+nd \d+:\d+:\d+|november \d+nd \d+:\d+:\d+|december \d+nd \d+:\d+:\d+ \
           january \d+rd \d+:\d+:\d+|february \d+rd \d+:\d+:\d+|march \d+rd \d+:\d+:\d+|april \d+rd \d+:\d+:\d+|may \d+rd \d+:\d+:\d+|june \d+rd \d+:\d+:\d+|july \d+rd \d+:\d+:\d+|august \d+rd \d+:\d+:\d+|september \d+rd \d+:\d+:\d+|october \d+rd \d+:\d+:\d+|november \d+rd \d+:\d+:\d+|december \d+rd \d+:\d+:\d+ \
           january \d+th \d+:\d+:\d+|february \d+th \d+:\d+:\d+|march \d+th \d+:\d+:\d+|april \d+th \d+:\d+:\d+|may \d+th \d+:\d+:\d+|june \d+th \d+:\d+:\d+|july \d+th \d+:\d+:\d+|august \d+th \d+:\d+:\d+|september \d+th \d+:\d+:\d+|october \d+th \d+:\d+:\d+|november \d+th \d+:\d+:\d+|december \d+th \d+:\d+:\d+ \
           january \d+ \d+:\d+:\d+|february \d+ \d+:\d+:\d+|march \d+ \d+:\d+:\d+|april \d+ \d+:\d+:\d+|may \d+ \d+:\d+:\d+|june \d+ \d+:\d+:\d+|july \d+ \d+:\d+:\d+|august \d+ \d+:\d+:\d+|september \d+ \d+:\d+:\d+|october \d+ \d+:\d+:\d+|november \d+ \d+:\d+:\d+|december \d+ \d+:\d+:\d+ "
        for match in re.findall(Month_DD_HMS, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            match_rep = re.sub("augu","august",match_rep)
            datetime_object = datetime.strptime(match_rep,"%B %d %H:%M:%S")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        Month_DD_HM = r"january \d+st \d+:\d+|february \d+st \d+:\d+|march \d+st \d+:\d+|april \d+st \d+:\d+|may \d+st \d+:\d+|june \d+st \d+:\d+|july \d+st \d+:\d+|august \d+st \d+:\d+|september \d+st \d+:\d+|october \d+st \d+:\d+|november \d+st \d+:\d+|december \d+st \d+:\d+ \
           january \d+nd \d+:\d+|february \d+nd \d+:\d+|march \d+nd \d+:\d+|april \d+nd \d+:\d+|may \d+nd \d+:\d+|june \d+nd \d+:\d+|july \d+nd \d+:\d+|august \d+nd \d+:\d+|september \d+nd \d+:\d+|october \d+nd \d+:\d+|november \d+nd \d+:\d+|december \d+nd \d+:\d+ \
           january \d+rd \d+:\d+|february \d+rd \d+:\d+|march \d+rd \d+:\d+|april \d+rd \d+:\d+|may \d+rd \d+:\d+|june \d+rd \d+:\d+|july \d+rd \d+:\d+|august \d+rd \d+:\d+|september \d+rd \d+:\d+|october \d+rd \d+:\d+|november \d+rd \d+:\d+|december \d+rd \d+:\d+ \
           january \d+th \d+:\d+|february \d+th \d+:\d+|march \d+th \d+:\d+|april \d+th \d+:\d+|may \d+th \d+:\d+|june \d+th \d+:\d+|july \d+th \d+:\d+|august \d+th \d+:\d+|september \d+th \d+:\d+|october \d+th \d+:\d+|november \d+th \d+:\d+|december \d+th \d+:\d+ \
           january \d+ \d+:\d+|february \d+ \d+:\d+|march \d+ \d+:\d+|april \d+ \d+:\d+|may \d+ \d+:\d+|june \d+ \d+:\d+|july \d+ \d+:\d+|august \d+ \d+:\d+|september \d+ \d+:\d+|october \d+ \d+:\d+|november \d+ \d+:\d+|december \d+ \d+:\d+ "
        for match in re.findall(Month_DD_HM, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            match_rep = re.sub("augu","august",match_rep)
            datetime_object = datetime.strptime(match_rep,"%B %d %H:%M")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
        Month_DD = r"january \d+st|february \d+st|march \d+st|april \d+st|may \d+st|june \d+st|july \d+st|august \d+st|september \d+st|october \d+st|november \d+st|december \d+st \
           january \d+nd|february \d+nd|march \d+nd|april \d+nd|may \d+nd|june \d+nd|july \d+nd|august \d+nd|september \d+nd|october \d+nd|november \d+nd|december \d+nd \
           january \d+rd|february \d+rd|march \d+rd|april \d+rd|may \d+rd|june \d+rd|july \d+rd|august \d+rd|september \d+rd|october \d+rd|november \d+rd|december \d+rd \
           january \d+th|february \d+th|march \d+th|april \d+th|may \d+th|june \d+th|july \d+th|august \d+th|september \d+th|october \d+th|november \d+th|december \d+th \
           january \d+|february \d+|march \d+|april \d+|may \d+|june \d+|july \d+|august \d+|september \d+|october \d+|november \d+|december \d+"
        for match in re.findall(Month_DD, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            match_rep = re.sub("augu","august",match_rep)
            datetime_object = datetime.strptime(match_rep,"%B %d")
            dateNewFormat = datetime_object.strftime("%B%#d")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        # No Space between month and date
        MonthDD_HMS = r"january\d+st \d+:\d+:\d+|february\d+st \d+:\d+:\d+|march\d+st \d+:\d+:\d+|april\d+st \d+:\d+:\d+|may\d+st \d+:\d+:\d+|june\d+st \d+:\d+:\d+|july\d+st \d+:\d+:\d+|august\d+st \d+:\d+:\d+|september\d+st \d+:\d+:\d+|october\d+st \d+:\d+:\d+|november\d+st \d+:\d+:\d+|december\d+st \d+:\d+:\d+ \
           january\d+nd \d+:\d+:\d+|february\d+nd \d+:\d+:\d+|march\d+nd \d+:\d+:\d+|april\d+nd \d+:\d+:\d+|may\d+nd \d+:\d+:\d+|june\d+nd \d+:\d+:\d+|july\d+nd \d+:\d+:\d+|august\d+nd \d+:\d+:\d+|september\d+nd \d+:\d+:\d+|october\d+nd \d+:\d+:\d+|november\d+nd \d+:\d+:\d+|december\d+nd \d+:\d+:\d+ \
           january\d+rd \d+:\d+:\d+|february\d+rd \d+:\d+:\d+|march\d+rd \d+:\d+:\d+|april\d+rd \d+:\d+:\d+|may\d+rd \d+:\d+:\d+|june\d+rd \d+:\d+:\d+|july\d+rd \d+:\d+:\d+|august\d+rd \d+:\d+:\d+|september\d+rd \d+:\d+:\d+|october\d+rd \d+:\d+:\d+|november\d+rd \d+:\d+:\d+|december\d+rd \d+:\d+:\d+ \
           january\d+th \d+:\d+:\d+|february\d+th \d+:\d+:\d+|march\d+th \d+:\d+:\d+|april\d+th \d+:\d+:\d+|may\d+th \d+:\d+:\d+|june\d+th \d+:\d+:\d+|july\d+th \d+:\d+:\d+|august\d+th \d+:\d+:\d+|september\d+th \d+:\d+:\d+|october\d+th \d+:\d+:\d+|november\d+th \d+:\d+:\d+|december\d+th \d+:\d+:\d+ \
           january\d+ \d+:\d+:\d+|february\d+ \d+:\d+:\d+|march\d+ \d+:\d+:\d+|april\d+ \d+:\d+:\d+|may\d+ \d+:\d+:\d+|june\d+ \d+:\d+:\d+|july\d+ \d+:\d+:\d+|august\d+ \d+:\d+:\d+|september\d+ \d+:\d+:\d+|october\d+ \d+:\d+:\d+|november\d+ \d+:\d+:\d+|december\d+ \d+:\d+:\d+ "
        for match in re.findall(MonthDD_HMS, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            match_rep = re.sub("augu","august",match_rep)
            datetime_object = datetime.strptime(match_rep,"%B%d %H:%M:%S")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        MonthDD_HM = r"january\d+st \d+:\d+|february\d+st \d+:\d+|march\d+st \d+:\d+|april\d+st \d+:\d+|may\d+st \d+:\d+|june\d+st \d+:\d+|july\d+st \d+:\d+|august\d+st \d+:\d+|september\d+st \d+:\d+|october\d+st \d+:\d+|november\d+st \d+:\d+|december\d+st \d+:\d+ \
           january\d+nd \d+:\d+|february\d+nd \d+:\d+|march\d+nd \d+:\d+|april\d+nd \d+:\d+|may\d+nd \d+:\d+|june\d+nd \d+:\d+|july\d+nd \d+:\d+|august\d+nd \d+:\d+|september\d+nd \d+:\d+|october\d+nd \d+:\d+|november\d+nd \d+:\d+|december\d+nd \d+:\d+ \
           january\d+rd \d+:\d+|february\d+rd \d+:\d+|march\d+rd \d+:\d+|april\d+rd \d+:\d+|may\d+rd \d+:\d+|june\d+rd \d+:\d+|july\d+rd \d+:\d+|august\d+rd \d+:\d+|september\d+rd \d+:\d+|october\d+rd \d+:\d+|november\d+rd \d+:\d+|december\d+rd \d+:\d+ \
           january\d+th \d+:\d+|february\d+th \d+:\d+|march\d+th \d+:\d+|april\d+th \d+:\d+|may\d+th \d+:\d+|june\d+th \d+:\d+|july\d+th \d+:\d+|august\d+th \d+:\d+|september\d+th \d+:\d+|october\d+th \d+:\d+|november\d+th \d+:\d+|december\d+th \d+:\d+ \
           january\d+ \d+:\d+|februaryruary\d+ \d+:\d+|march\d+ \d+:\d+|april\d+ \d+:\d+|may\d+ \d+:\d+|june\d+ \d+:\d+|july\d+ \d+:\d+|august\d+ \d+:\d+|september\d+ \d+:\d+|october\d+ \d+:\d+|november\d+ \d+:\d+|december\d+ \d+:\d+ "
        for match in re.findall(MonthDD_HM, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            match_rep = re.sub("augu","august",match_rep)
            datetime_object = datetime.strptime(match_rep,"%B%d %H:%M")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        MonthDD = r"january\d+st|february\d+st|march\d+st|april\d+st|may\d+st|june\d+st|july\d+st|august\d+st|september\d+st|october\d+st|november\d+st|december\d+st \
           january\d+nd|february\d+nd|march\d+nd|april\d+nd|may\d+nd|june\d+nd|july\d+nd|august\d+nd|september\d+nd|october\d+nd|november\d+nd|december\d+nd \
           january\d+rd|february\d+rd|march\d+rd|april\d+rd|may\d+rd|june\d+rd|july\d+rd|august\d+rd|september\d+rd|october\d+rd|november\d+rd|december\d+rd \
           january\d+th|february\d+th|march\d+th|april\d+th|may\d+th|june\d+th|july\d+th|august\d+th|september\d+th|october\d+th|november\d+th|december\d+th \
           january\d+|february\d+|march\d+|april\d+|may\d+|june\d+|july\d+|august\d+|september\d+|october\d+|november\d+|december\d+ "
        for match in re.findall(MonthDD, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            match_rep = re.sub("augu","august",match_rep)
            datetime_object = datetime.strptime(match_rep,"%B%d")
            dateNewFormat = datetime_object.strftime("%B%#d")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        # Replace 2/29 to february29
        for match in re.findall("2/29", TextwithDates):
            TextwithDates = re.sub(match, "February29", TextwithDates)

        # First convert YYYY to YY
        for match in re.findall(r'/\d{4}', TextwithDates):
            match = re.sub('/','' , match)
            #convert match to new format
            datetime_object = datetime.strptime(match, "%Y")
            dateNewFormat = datetime_object.strftime("%y")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        # convert %m/%d/%y formatted texts to %m/%d
        for match in re.findall(r'\d+/\d+/\d+', TextwithDates):
            #convert match to new format
            datetime_object = datetime.strptime(match, "%m/%d/%y")
            dateNewFormat = datetime_object.strftime("%#m/%#d")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        # convert %m/%d %H:%M:%S formatted texts to %B%d_%H:%M:%S
        for match in re.findall(r'\d+/\d+ \d+:\d+:\d+', TextwithDates):
            datetime_object = datetime.strptime(match, "%m/%d %H:%M:%S")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        # convert %m/%d %H:%M formatted texts to %B%d_%H:%M
        for match in re.findall(r'\d+/\d+ \d+:\d+', TextwithDates):
            datetime_object = datetime.strptime(match, "%m/%d %H:%M")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        # convert %m/%d formatted texts to %B%d
        for match in re.findall(r'\d+/\d+', TextwithDates):
            datetime_object = datetime.strptime(match, "%m/%d")
            dateNewFormat = datetime_object.strftime("%B%#d")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
        return(TextwithDates)
    except Exception:
        pass
    
# MonthDD = r"January\d+|February\d+|March\d+|April\d+|May\d+|June\d+|July\d+|August\d+|September\d+|October\d+|November\d+|December\d+"    

In [11]:
# Check for errors 
ClinicalNotes_error = ClinicalNotes.copy()
ClinicalNotes_error['Text_error'] = ClinicalNotes_error.apply(lambda x: ErrorINChangeDateTimeFormat(x.Text), axis=1)

# Extract cases where there are no errors
ClinicalNotes_NoDateErrors = ClinicalNotes_error[pd.notnull(ClinicalNotes_error.Text_error)]
ClinicalNotes_NoDateErrors = ClinicalNotes_NoDateErrors.drop(['Text_error'],axis=1)

In [ ]:
ClinicalNotes_NoDateErrors.head()

In [7]:
# Returns exception i.e error
def ErrorsINChangeDateTimeFormat(TextwithDates):
    try:
        # Replace sept with sep            
        for match in re.findall("sept \d+", TextwithDates):
            match_rep = re.sub('sept','sep' ,match)
            TextwithDates = re.sub(match, match_rep, TextwithDates)
            
        # Convert jan & january independently     
        suffix = r"st|nd|rd|th"
    
        # Extract Month_Year
        Month_Year = "january \d{4}|february \d{4}|march \d{4}|april \d{4}|may \d{4}|june \d{4}|july \d{4}|august \d{4}|september \d{4}|october \d{4}|november \d{4}|december \d{4}"
        for match in re.findall(Month_Year, TextwithDates):
            datetime_object = datetime.strptime(match,"%B %Y")
            dateNewFormat = datetime_object.strftime("%Y%B")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
        # Mon_Year
        Mon_Year = "jan \d{4}|feb \d{4}|mar \d{4}|apr \d{4}|may \d{4}|jun \d{4}|jul \d{4}|aug \d{4}|sep \d{4}|oct \d{4}|nov \d{4}|dec \d{4}"
        for match in re.findall(Mon_Year, TextwithDates):
            datetime_object = datetime.strptime(match,"%b %Y")
            dateNewFormat = datetime_object.strftime("%Y%B")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        # No Space between month and date
        MonDD_HMS = r"jan\d+st \d+:\d+:\d+|feb\d+st \d+:\d+:\d+|mar\d+st \d+:\d+:\d+|apr\d+st \d+:\d+:\d+|may\d+st \d+:\d+:\d+|jun\d+st \d+:\d+:\d+|jul\d+st \d+:\d+:\d+|aug\d+st \d+:\d+:\d+|sep\d+st \d+:\d+:\d+|oct\d+st \d+:\d+:\d+|nov\d+st \d+:\d+:\d+|dec\d+st \d+:\d+:\d+ \
           jan\d+nd \d+:\d+:\d+|feb\d+nd \d+:\d+:\d+|mar\d+nd \d+:\d+:\d+|apr\d+nd \d+:\d+:\d+|may\d+nd \d+:\d+:\d+|jun\d+nd \d+:\d+:\d+|jul\d+nd \d+:\d+:\d+|aug\d+nd \d+:\d+:\d+|sep\d+nd \d+:\d+:\d+|oct\d+nd \d+:\d+:\d+|nov\d+nd \d+:\d+:\d+|dec\d+nd \d+:\d+:\d+ \
           jan\d+rd \d+:\d+:\d+|feb\d+rd \d+:\d+:\d+|mar\d+rd \d+:\d+:\d+|apr\d+rd \d+:\d+:\d+|may\d+rd \d+:\d+:\d+|jun\d+rd \d+:\d+:\d+|jul\d+rd \d+:\d+:\d+|aug\d+rd \d+:\d+:\d+|sep\d+rd \d+:\d+:\d+|oct\d+rd \d+:\d+:\d+|nov\d+rd \d+:\d+:\d+|dec\d+rd \d+:\d+:\d+ \
           jan\d+th \d+:\d+:\d+|feb\d+th \d+:\d+:\d+|mar\d+th \d+:\d+:\d+|apr\d+th \d+:\d+:\d+|may\d+th \d+:\d+:\d+|jun\d+th \d+:\d+:\d+|jul\d+th \d+:\d+:\d+|aug\d+th \d+:\d+:\d+|sep\d+th \d+:\d+:\d+|oct\d+th \d+:\d+:\d+|nov\d+th \d+:\d+:\d+|dec\d+th \d+:\d+:\d+ \
           jan\d+ \d+:\d+:\d+|feb\d+ \d+:\d+:\d+|mar\d+ \d+:\d+:\d+|apr\d+ \d+:\d+:\d+|may\d+ \d+:\d+:\d+|jun\d+ \d+:\d+:\d+|jul\d+ \d+:\d+:\d+|aug\d+ \d+:\d+:\d+|sep\d+ \d+:\d+:\d+|oct\d+ \d+:\d+:\d+|nov\d+ \d+:\d+:\d+|dec\d+ \d+:\d+:\d+"
        for match in re.findall(MonDD_HMS, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            datetime_object = datetime.strptime(match_rep,"%b%d %H:%M:%S")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
        MonDD_HM = r"jan\d+st \d+:\d+|feb\d+st \d+:\d+|mar\d+st \d+:\d+|apr\d+st \d+:\d+|may\d+st \d+:\d+|jun\d+st \d+:\d+|jul\d+st \d+:\d+|aug\d+st \d+:\d+|sep\d+st \d+:\d+|oct\d+st \d+:\d+|nov\d+st \d+:\d+|dec\d+st \d+:\d+ \
           jan\d+nd \d+:\d+|feb\d+nd \d+:\d+|mar\d+nd \d+:\d+|apr\d+nd \d+:\d+|may\d+nd \d+:\d+|jun\d+nd \d+:\d+|jul\d+nd \d+:\d+|aug\d+nd \d+:\d+|sep\d+nd \d+:\d+|oct\d+nd \d+:\d+|nov\d+nd \d+:\d+|dec\d+nd \d+:\d+ \
           jan\d+rd \d+:\d+|feb\d+rd \d+:\d+|mar\d+rd \d+:\d+|apr\d+rd \d+:\d+|may\d+rd \d+:\d+|jun\d+rd \d+:\d+|jul\d+rd \d+:\d+|aug\d+rd \d+:\d+|sep\d+rd \d+:\d+|oct\d+rd \d+:\d+|nov\d+rd \d+:\d+|dec\d+rd \d+:\d+ \
           jan\d+th \d+:\d+|feb\d+th \d+:\d+|mar\d+th \d+:\d+|apr\d+th \d+:\d+|may\d+th \d+:\d+|jun\d+th \d+:\d+|jul\d+th \d+:\d+|aug\d+th \d+:\d+|sep\d+th \d+:\d+|oct\d+th \d+:\d+|nov\d+th \d+:\d+|dec\d+th \d+:\d+ \
           jan\d+ \d+:\d+|feb\d+ \d+:\d+|mar\d+ \d+:\d+|apr\d+ \d+:\d+|may\d+ \d+:\d+|jun\d+ \d+:\d+|jul\d+ \d+:\d+|aug\d+ \d+:\d+|sep\d+ \d+:\d+|oct\d+ \d+:\d+|nov\d+ \d+:\d+|dec\d+ \d+:\d+"
           
        for match in re.findall(MonDD_HM, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            datetime_object = datetime.strptime(match_rep,"%b%d %H:%M")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
        MonDD = r"jan\d+st|feb\d+st|mar\d+st|apr\d+st|may\d+st|jun\d+st|jul\d+st|aug\d+st|sep\d+st|oct\d+st|nov\d+st|dec\d+st \
           jan\d+nd|feb\d+nd|mar\d+nd|apr\d+nd|may\d+nd|jun\d+nd|jul\d+nd|aug\d+nd|sep\d+nd|oct\d+nd|nov\d+nd|dec\d+nd \
           jan\d+rd|feb\d+rd|mar\d+rd|apr\d+rd|may\d+rd|jun\d+rd|jul\d+rd|aug\d+rd|sep\d+rd|oct\d+rd|nov\d+rd|dec\d+rd \
           jan\d+th|feb\d+th|mar\d+th|apr\d+th|may\d+th|jun\d+th|jul\d+th|aug\d+th|sep\d+th|oct\d+th|nov\d+th|dec\d+th \
           jan\d+|feb\d+|mar\d+|apr\d+|may\d+|jun\d+|jul\d+|aug\d+|sep\d+|oct\d+|nov\d+|dec\d+"
        for match in re.findall(MonDD, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            datetime_object = datetime.strptime(match_rep,"%b%d")
            dateNewFormat = datetime_object.strftime("%B%#d")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        # Space between month and date
        Mon_DD_HMS = r"jan \d+st \d+:\d+:\d+|feb \d+st \d+:\d+:\d+|mar \d+st \d+:\d+:\d+|apr \d+st \d+:\d+:\d+|may \d+st \d+:\d+:\d+|jun \d+st \d+:\d+:\d+|jul \d+st \d+:\d+:\d+|aug \d+st \d+:\d+:\d+|sep \d+st \d+:\d+:\d+|oct \d+st \d+:\d+:\d+|nov \d+st \d+:\d+:\d+|dec \d+st \d+:\d+:\d+ \
           jan \d+nd \d+:\d+:\d+|feb \d+nd \d+:\d+:\d+|mar \d+nd \d+:\d+:\d+|apr \d+nd \d+:\d+:\d+|may \d+nd \d+:\d+:\d+|jun \d+nd \d+:\d+:\d+|jul \d+nd \d+:\d+:\d+|aug \d+nd \d+:\d+:\d+|sep \d+nd \d+:\d+:\d+|oct \d+nd \d+:\d+:\d+|nov \d+nd \d+:\d+:\d+|dec \d+nd \d+:\d+:\d+ \
           jan \d+rd \d+:\d+:\d+|feb \d+rd \d+:\d+:\d+|mar \d+rd \d+:\d+:\d+|apr \d+rd \d+:\d+:\d+|may \d+rd \d+:\d+:\d+|jun \d+rd \d+:\d+:\d+|jul \d+rd \d+:\d+:\d+|aug \d+rd \d+:\d+:\d+|sep \d+rd \d+:\d+:\d+|oct \d+rd \d+:\d+:\d+|nov \d+rd \d+:\d+:\d+|dec \d+rd \d+:\d+:\d+ \
           jan \d+th \d+:\d+:\d+|feb \d+th \d+:\d+:\d+|mar \d+th \d+:\d+:\d+|apr \d+th \d+:\d+:\d+|may \d+th \d+:\d+:\d+|jun \d+th \d+:\d+:\d+|jul \d+th \d+:\d+:\d+|aug \d+th \d+:\d+:\d+|sep \d+th \d+:\d+:\d+|oct \d+th \d+:\d+:\d+|nov \d+th \d+:\d+:\d+|dec \d+th \d+:\d+:\d+ \
           jan \d+ \d+:\d+:\d+|feb \d+ \d+:\d+:\d+|mar \d+ \d+:\d+:\d+|apr \d+ \d+:\d+:\d+|may \d+ \d+:\d+:\d+|jun \d+ \d+:\d+:\d+|jul \d+ \d+:\d+:\d+|aug \d+ \d+:\d+:\d+|sep \d+ \d+:\d+:\d+|oct \d+ \d+:\d+:\d+|nov \d+ \d+:\d+:\d+|dec \d+ \d+:\d+:\d+"
        for match in re.findall(Mon_DD_HMS, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            datetime_object = datetime.strptime(match_rep,"%b %d %H:%M:%S")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
        Mon_DD_HM = r"jan \d+st \d+:\d+|feb \d+st \d+:\d+|mar \d+st \d+:\d+|apr \d+st \d+:\d+|may \d+st \d+:\d+|jun \d+st \d+:\d+|jul \d+st \d+:\d+|aug \d+st \d+:\d+|sep \d+st \d+:\d+|oct \d+st \d+:\d+|nov \d+st \d+:\d+|dec \d+st \d+:\d+ \
           jan \d+nd \d+:\d+|feb \d+nd \d+:\d+|mar \d+nd \d+:\d+|apr \d+nd \d+:\d+|may \d+nd \d+:\d+|jun \d+nd \d+:\d+|jul \d+nd \d+:\d+|aug \d+nd \d+:\d+|sep \d+nd \d+:\d+|oct \d+nd \d+:\d+|nov \d+nd \d+:\d+|dec \d+nd \d+:\d+ \
           jan \d+rd \d+:\d+|feb \d+rd \d+:\d+|mar \d+rd \d+:\d+|apr \d+rd \d+:\d+|may \d+rd \d+:\d+|jun \d+rd \d+:\d+|jul \d+rd \d+:\d+|aug \d+rd \d+:\d+|sep \d+rd \d+:\d+|oct \d+rd \d+:\d+|nov \d+rd \d+:\d+|dec \d+rd \d+:\d+ \
           jan \d+th \d+:\d+|feb \d+th \d+:\d+|mar \d+th \d+:\d+|apr \d+th \d+:\d+|may \d+th \d+:\d+|jun \d+th \d+:\d+|jul \d+th \d+:\d+|aug \d+th \d+:\d+|sep \d+th \d+:\d+|oct \d+th \d+:\d+|nov \d+th \d+:\d+|dec \d+th \d+:\d+ \
           jan \d+ \d+:\d+|feb \d+ \d+:\d+|mar \d+ \d+:\d+|apr \d+ \d+:\d+|may \d+ \d+:\d+|jun \d+ \d+:\d+|jul \d+ \d+:\d+|aug \d+ \d+:\d+|sep \d+ \d+:\d+|oct \d+ \d+:\d+|nov \d+ \d+:\d+|dec \d+ \d+:\d+"
        for match in re.findall(Mon_DD_HM, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            datetime_object = datetime.strptime(match_rep,"%b %d %H:%M")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        Mon_DD = r"jan \d+st|feb \d+st|mar \d+st|apr \d+st|may \d+st|jun \d+st|jul \d+st|aug \d+st|sep \d+st|oct \d+st|nov \d+st|dec \d+st \
           jan \d+nd|feb \d+nd|mar \d+nd|apr \d+nd|may \d+nd|jun \d+nd|jul \d+nd|aug \d+nd|sep \d+nd|oct \d+nd|nov \d+nd|dec \d+nd \
           jan \d+rd|feb \d+rd|mar \d+rd|apr \d+rd|may \d+rd|jun \d+rd|jul \d+rd|aug \d+rd|sep \d+rd|oct \d+rd|nov \d+rd|dec \d+rd \
           jan \d+th|feb \d+th|mar \d+th|apr \d+th|may \d+th|jun \d+th|jul \d+th|aug \d+th|sep \d+th|oct \d+th|nov \d+th|dec \d+th \
           jan \d+|feb \d+|mar \d+|apr \d+|may \d+|jun \d+|jul \d+|aug \d+|sep \d+|oct \d+|nov \d+|dec \d+"
        for match in re.findall(Mon_DD, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            datetime_object = datetime.strptime(match_rep,"%b %d")
            dateNewFormat = datetime_object.strftime("%B%#d")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        # Space between month and date
        Month_DD_HMS = r"january \d+st \d+:\d+:\d+|february \d+st \d+:\d+:\d+|march \d+st \d+:\d+:\d+|april \d+st \d+:\d+:\d+|may \d+st \d+:\d+:\d+|june \d+st \d+:\d+:\d+|july \d+st \d+:\d+:\d+|august \d+st \d+:\d+:\d+|september \d+st \d+:\d+:\d+|october \d+st \d+:\d+:\d+|november \d+st \d+:\d+:\d+|december \d+st \d+:\d+:\d+ \
           january \d+nd \d+:\d+:\d+|february \d+nd \d+:\d+:\d+|march \d+nd \d+:\d+:\d+|april \d+nd \d+:\d+:\d+|may \d+nd \d+:\d+:\d+|june \d+nd \d+:\d+:\d+|july \d+nd \d+:\d+:\d+|august \d+nd \d+:\d+:\d+|september \d+nd \d+:\d+:\d+|october \d+nd \d+:\d+:\d+|november \d+nd \d+:\d+:\d+|december \d+nd \d+:\d+:\d+ \
           january \d+rd \d+:\d+:\d+|february \d+rd \d+:\d+:\d+|march \d+rd \d+:\d+:\d+|april \d+rd \d+:\d+:\d+|may \d+rd \d+:\d+:\d+|june \d+rd \d+:\d+:\d+|july \d+rd \d+:\d+:\d+|august \d+rd \d+:\d+:\d+|september \d+rd \d+:\d+:\d+|october \d+rd \d+:\d+:\d+|november \d+rd \d+:\d+:\d+|december \d+rd \d+:\d+:\d+ \
           january \d+th \d+:\d+:\d+|february \d+th \d+:\d+:\d+|march \d+th \d+:\d+:\d+|april \d+th \d+:\d+:\d+|may \d+th \d+:\d+:\d+|june \d+th \d+:\d+:\d+|july \d+th \d+:\d+:\d+|august \d+th \d+:\d+:\d+|september \d+th \d+:\d+:\d+|october \d+th \d+:\d+:\d+|november \d+th \d+:\d+:\d+|december \d+th \d+:\d+:\d+ \
           january \d+ \d+:\d+:\d+|february \d+ \d+:\d+:\d+|march \d+ \d+:\d+:\d+|april \d+ \d+:\d+:\d+|may \d+ \d+:\d+:\d+|june \d+ \d+:\d+:\d+|july \d+ \d+:\d+:\d+|august \d+ \d+:\d+:\d+|september \d+ \d+:\d+:\d+|october \d+ \d+:\d+:\d+|november \d+ \d+:\d+:\d+|december \d+ \d+:\d+:\d+ "
        for match in re.findall(Month_DD_HMS, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            match_rep = re.sub("augu","august",match_rep)
            datetime_object = datetime.strptime(match_rep,"%B %d %H:%M:%S")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        Month_DD_HM = r"january \d+st \d+:\d+|february \d+st \d+:\d+|march \d+st \d+:\d+|april \d+st \d+:\d+|may \d+st \d+:\d+|june \d+st \d+:\d+|july \d+st \d+:\d+|august \d+st \d+:\d+|september \d+st \d+:\d+|october \d+st \d+:\d+|november \d+st \d+:\d+|december \d+st \d+:\d+ \
           january \d+nd \d+:\d+|february \d+nd \d+:\d+|march \d+nd \d+:\d+|april \d+nd \d+:\d+|may \d+nd \d+:\d+|june \d+nd \d+:\d+|july \d+nd \d+:\d+|august \d+nd \d+:\d+|september \d+nd \d+:\d+|october \d+nd \d+:\d+|november \d+nd \d+:\d+|december \d+nd \d+:\d+ \
           january \d+rd \d+:\d+|february \d+rd \d+:\d+|march \d+rd \d+:\d+|april \d+rd \d+:\d+|may \d+rd \d+:\d+|june \d+rd \d+:\d+|july \d+rd \d+:\d+|august \d+rd \d+:\d+|september \d+rd \d+:\d+|october \d+rd \d+:\d+|november \d+rd \d+:\d+|december \d+rd \d+:\d+ \
           january \d+th \d+:\d+|february \d+th \d+:\d+|march \d+th \d+:\d+|april \d+th \d+:\d+|may \d+th \d+:\d+|june \d+th \d+:\d+|july \d+th \d+:\d+|august \d+th \d+:\d+|september \d+th \d+:\d+|october \d+th \d+:\d+|november \d+th \d+:\d+|december \d+th \d+:\d+ \
           january \d+ \d+:\d+|february \d+ \d+:\d+|march \d+ \d+:\d+|april \d+ \d+:\d+|may \d+ \d+:\d+|june \d+ \d+:\d+|july \d+ \d+:\d+|august \d+ \d+:\d+|september \d+ \d+:\d+|october \d+ \d+:\d+|november \d+ \d+:\d+|december \d+ \d+:\d+ "
        for match in re.findall(Month_DD_HM, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            match_rep = re.sub("augu","august",match_rep)
            datetime_object = datetime.strptime(match_rep,"%B %d %H:%M")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
        Month_DD = r"january \d+st|february \d+st|march \d+st|april \d+st|may \d+st|june \d+st|july \d+st|august \d+st|september \d+st|october \d+st|november \d+st|december \d+st \
           january \d+nd|february \d+nd|march \d+nd|april \d+nd|may \d+nd|june \d+nd|july \d+nd|august \d+nd|september \d+nd|october \d+nd|november \d+nd|december \d+nd \
           january \d+rd|february \d+rd|march \d+rd|april \d+rd|may \d+rd|june \d+rd|july \d+rd|august \d+rd|september \d+rd|october \d+rd|november \d+rd|december \d+rd \
           january \d+th|february \d+th|march \d+th|april \d+th|may \d+th|june \d+th|july \d+th|august \d+th|september \d+th|october \d+th|november \d+th|december \d+th \
           january \d+|february \d+|march \d+|april \d+|may \d+|june \d+|july \d+|august \d+|september \d+|october \d+|november \d+|december \d+"
        for match in re.findall(Month_DD, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            match_rep = re.sub("augu","august",match_rep)
            datetime_object = datetime.strptime(match_rep,"%B %d")
            dateNewFormat = datetime_object.strftime("%B%#d")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        # No Space between month and date
        MonthDD_HMS = r"january\d+st \d+:\d+:\d+|february\d+st \d+:\d+:\d+|march\d+st \d+:\d+:\d+|april\d+st \d+:\d+:\d+|may\d+st \d+:\d+:\d+|june\d+st \d+:\d+:\d+|july\d+st \d+:\d+:\d+|august\d+st \d+:\d+:\d+|september\d+st \d+:\d+:\d+|october\d+st \d+:\d+:\d+|november\d+st \d+:\d+:\d+|december\d+st \d+:\d+:\d+ \
           january\d+nd \d+:\d+:\d+|february\d+nd \d+:\d+:\d+|march\d+nd \d+:\d+:\d+|april\d+nd \d+:\d+:\d+|may\d+nd \d+:\d+:\d+|june\d+nd \d+:\d+:\d+|july\d+nd \d+:\d+:\d+|august\d+nd \d+:\d+:\d+|september\d+nd \d+:\d+:\d+|october\d+nd \d+:\d+:\d+|november\d+nd \d+:\d+:\d+|december\d+nd \d+:\d+:\d+ \
           january\d+rd \d+:\d+:\d+|february\d+rd \d+:\d+:\d+|march\d+rd \d+:\d+:\d+|april\d+rd \d+:\d+:\d+|may\d+rd \d+:\d+:\d+|june\d+rd \d+:\d+:\d+|july\d+rd \d+:\d+:\d+|august\d+rd \d+:\d+:\d+|september\d+rd \d+:\d+:\d+|october\d+rd \d+:\d+:\d+|november\d+rd \d+:\d+:\d+|december\d+rd \d+:\d+:\d+ \
           january\d+th \d+:\d+:\d+|february\d+th \d+:\d+:\d+|march\d+th \d+:\d+:\d+|april\d+th \d+:\d+:\d+|may\d+th \d+:\d+:\d+|june\d+th \d+:\d+:\d+|july\d+th \d+:\d+:\d+|august\d+th \d+:\d+:\d+|september\d+th \d+:\d+:\d+|october\d+th \d+:\d+:\d+|november\d+th \d+:\d+:\d+|december\d+th \d+:\d+:\d+ \
           january\d+ \d+:\d+:\d+|february\d+ \d+:\d+:\d+|march\d+ \d+:\d+:\d+|april\d+ \d+:\d+:\d+|may\d+ \d+:\d+:\d+|june\d+ \d+:\d+:\d+|july\d+ \d+:\d+:\d+|august\d+ \d+:\d+:\d+|september\d+ \d+:\d+:\d+|october\d+ \d+:\d+:\d+|november\d+ \d+:\d+:\d+|december\d+ \d+:\d+:\d+ "
        for match in re.findall(MonthDD_HMS, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            match_rep = re.sub("augu","august",match_rep)
            datetime_object = datetime.strptime(match_rep,"%B%d %H:%M:%S")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        MonthDD_HM = r"january\d+st \d+:\d+|february\d+st \d+:\d+|march\d+st \d+:\d+|april\d+st \d+:\d+|may\d+st \d+:\d+|june\d+st \d+:\d+|july\d+st \d+:\d+|august\d+st \d+:\d+|september\d+st \d+:\d+|october\d+st \d+:\d+|november\d+st \d+:\d+|december\d+st \d+:\d+ \
           january\d+nd \d+:\d+|february\d+nd \d+:\d+|march\d+nd \d+:\d+|april\d+nd \d+:\d+|may\d+nd \d+:\d+|june\d+nd \d+:\d+|july\d+nd \d+:\d+|august\d+nd \d+:\d+|september\d+nd \d+:\d+|october\d+nd \d+:\d+|november\d+nd \d+:\d+|december\d+nd \d+:\d+ \
           january\d+rd \d+:\d+|february\d+rd \d+:\d+|march\d+rd \d+:\d+|april\d+rd \d+:\d+|may\d+rd \d+:\d+|june\d+rd \d+:\d+|july\d+rd \d+:\d+|august\d+rd \d+:\d+|september\d+rd \d+:\d+|october\d+rd \d+:\d+|november\d+rd \d+:\d+|december\d+rd \d+:\d+ \
           january\d+th \d+:\d+|february\d+th \d+:\d+|march\d+th \d+:\d+|april\d+th \d+:\d+|may\d+th \d+:\d+|june\d+th \d+:\d+|july\d+th \d+:\d+|august\d+th \d+:\d+|september\d+th \d+:\d+|october\d+th \d+:\d+|november\d+th \d+:\d+|december\d+th \d+:\d+ \
           january\d+ \d+:\d+|februaryruary\d+ \d+:\d+|march\d+ \d+:\d+|april\d+ \d+:\d+|may\d+ \d+:\d+|june\d+ \d+:\d+|july\d+ \d+:\d+|august\d+ \d+:\d+|september\d+ \d+:\d+|october\d+ \d+:\d+|november\d+ \d+:\d+|december\d+ \d+:\d+ "
        for match in re.findall(MonthDD_HM, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            match_rep = re.sub("augu","august",match_rep)
            datetime_object = datetime.strptime(match_rep,"%B%d %H:%M")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        MonthDD = r"january\d+st|february\d+st|march\d+st|april\d+st|may\d+st|june\d+st|july\d+st|august\d+st|september\d+st|october\d+st|november\d+st|december\d+st \
           january\d+nd|february\d+nd|march\d+nd|april\d+nd|may\d+nd|june\d+nd|july\d+nd|august\d+nd|september\d+nd|october\d+nd|november\d+nd|december\d+nd \
           january\d+rd|february\d+rd|march\d+rd|april\d+rd|may\d+rd|june\d+rd|july\d+rd|august\d+rd|september\d+rd|october\d+rd|november\d+rd|december\d+rd \
           january\d+th|february\d+th|march\d+th|april\d+th|may\d+th|june\d+th|july\d+th|august\d+th|september\d+th|october\d+th|november\d+th|december\d+th \
           january\d+|february\d+|march\d+|april\d+|may\d+|june\d+|july\d+|august\d+|september\d+|october\d+|november\d+|december\d+ "
        for match in re.findall(MonthDD, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            match_rep = re.sub("augu","august",match_rep)
            datetime_object = datetime.strptime(match_rep,"%B%d")
            dateNewFormat = datetime_object.strftime("%B%#d")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        # Replace 2/29 to february29
        for match in re.findall("2/29", TextwithDates):
            TextwithDates = re.sub(match, "February29", TextwithDates)

        # First convert YYYY to YY
        for match in re.findall(r'/\d{4}', TextwithDates):
            match = re.sub('/','' , match)
            #convert match to new format
            datetime_object = datetime.strptime(match, "%Y")
            dateNewFormat = datetime_object.strftime("%y")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        # convert %m/%d/%y formatted texts to %m/%d
        for match in re.findall(r'\d+/\d+/\d+', TextwithDates):
            #convert match to new format
            datetime_object = datetime.strptime(match, "%m/%d/%y")
            dateNewFormat = datetime_object.strftime("%#m/%#d")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        # convert %m/%d %H:%M:%S formatted texts to %B%d_%H:%M:%S
        for match in re.findall(r'\d+/\d+ \d+:\d+:\d+', TextwithDates):
            datetime_object = datetime.strptime(match, "%m/%d %H:%M:%S")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        # convert %m/%d %H:%M formatted texts to %B%d_%H:%M
        for match in re.findall(r'\d+/\d+ \d+:\d+', TextwithDates):
            datetime_object = datetime.strptime(match, "%m/%d %H:%M")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        # convert %m/%d formatted texts to %B%d
        for match in re.findall(r'\d+/\d+', TextwithDates):
            datetime_object = datetime.strptime(match, "%m/%d")
            dateNewFormat = datetime_object.strftime("%B%#d")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
        return(TextwithDates)
    except Exception as e:
        return(e)
    
# MonthDD = r"January\d+|February\d+|March\d+|April\d+|May\d+|June\d+|July\d+|August\d+|September\d+|October\d+|November\d+|December\d+"    

In [44]:
# Text_error is missing if an error is present while processing functions
ClinicalNotes_error = ClinicalNotes_error[pd.isnull(ClinicalNotes_error.Text_error)]

# Text_error here produces the error for the above data set
ClinicalNotes_error['Text_error'] = ClinicalNotes_error.apply(lambda x: ErrorsINChangeDateTimeFormat(x.Text), axis=1)
ClinicalNotes_error.to_excel('Py DataFrames/ClinicalNotes_errors.xlsx',index=False)

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [ ]:
# textToSearch = "date of service from 8/6 to 9/8/19 "
# textToSearch = "date of service from 8/6 to 9/8/19 and 12/8/2019"
# textToSearch = "date of service from 8/6 to 9/8/19 and 12/8/2019 and september 6th 2019"
# textToSearch = "date of service from 8/6 to 9/8/19 and 12/8/2019 and september 6th 2019 in 101.4 as 1400" # year 
# textToSearch = "date of service from 8/6 to 9/8/19 and 12/8/2019 and september 6th 2019 in 101.4 as @1400"
# textToSearch = "date of service from 8/6 to 9/8/19 and 12/8/2019 and september 6th 2019 in 101.4 as 6000"
# textToSearch = "date of service from sep08 to 8/6/19 to 9/8/19 to 08/05/19 17:26:00 CDT and not 6/7"
# textToSearch = "08/05/19 17:26:00 CDT 08/05 4:05 9/8/19 and 12/8/2019 temp 101 and september 6 2019 march 8th xxx jan10th xyz jan 8th  na 140 (aug 07) 143 (aug 06) 141 (aug 05) " # 
# textToSearch = "08/05/19 17:26:00 CDT 08/05 4:05 9/8/19 and 12/8/2019 temp 101 and september 6 2019 march 8th xxx jan10th xyz jan 8th  na 140 (aug 07) 143 (aug 06) 141 (aug 05) " # 
# textToSearch = "scr remains stable 1.6-1.9. ply holding at 180s, ast/alt 76/51" # 
# textToSearch = "16/6 per prog note: her work up for infectious colitis thus far has been negative" #If date is in dd/mm format the code ignores or doesn't convert 
# textToSearch = "9/26-\nmica 9/27-\nceftriaxone 9/24-9/26\nmero 9/229/24\n\n9/27 id on board" #In this case, 9/229/24 it is supposed to be 9/22 and 9/24 but since there is no gap etc between the two it is considered as an error
# textToSearch = "percocet 5/325\n\npmh: cad, chf, gerd, htn" 
# # In row 4722 - looks like only of its date is written in dd/mm format
# # In row 22572 - check again
# # In row 33745 - 169382/1283521

# textToSearch = ClinicalNotes[ClinicalNotes.Row==3099].Text
# textToSearch

# ClinicalNotes_WBC['wbcRows'] = ClinicalNotes_WBC['Text_DateTimeFormatted'].str.findall(r'wbc')
# str(ClinicalNotes[ClinicalNotes.Row==984].Text).replace('"', "") - string error
# re.findall(r'\d+/\d+/\d+', textToSearch)

# textToSearch = str(ClinicalNotes[ClinicalNotes.Row==984].Text).replace('"', "")
# ErrorsINChangeDateTimeFormat(textToSearch)

# ChangeDateFormat(textToSearch)
# ErrorsINChangeDateTimeFormat(textToSearch)

# MonthDD = r"January\d+|February\d+|March\d+|April\d+|May\d+|June\d+|July\d+|August\d+|September\d+|October\d+|November\d+|December\d+"
# for match in re.findall(MonthDD, textToSearch):
#     print(match)

# ClinicalNotes[((ClinicalNotes['Text'].str.findall(r'\\')).str.len())!=0]

In [8]:
def ChangeDateTimeFormat(TextwithDates):
    try:
        # Replace sept with sep            
        for match in re.findall("sept \d+", TextwithDates):
            match_rep = re.sub('sept','sep' ,match)
            TextwithDates = re.sub(match, match_rep, TextwithDates)
            
        # Convert jan & january independently     
        suffix = r"st|nd|rd|th"
    
        # Extract Month_Year
        Month_Year = "january \d{4}|february \d{4}|march \d{4}|april \d{4}|may \d{4}|june \d{4}|july \d{4}|august \d{4}|september \d{4}|october \d{4}|november \d{4}|december \d{4}"
        for match in re.findall(Month_Year, TextwithDates):
            datetime_object = datetime.strptime(match,"%B %Y")
            dateNewFormat = datetime_object.strftime("%Y%B")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
        # Mon_Year
        Mon_Year = "jan \d{4}|feb \d{4}|mar \d{4}|apr \d{4}|may \d{4}|jun \d{4}|jul \d{4}|aug \d{4}|sep \d{4}|oct \d{4}|nov \d{4}|dec \d{4}"
        for match in re.findall(Mon_Year, TextwithDates):
            datetime_object = datetime.strptime(match,"%b %Y")
            dateNewFormat = datetime_object.strftime("%Y%B")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        # No Space between month and date
        MonDD_HMS = r"jan\d+st \d+:\d+:\d+|feb\d+st \d+:\d+:\d+|mar\d+st \d+:\d+:\d+|apr\d+st \d+:\d+:\d+|may\d+st \d+:\d+:\d+|jun\d+st \d+:\d+:\d+|jul\d+st \d+:\d+:\d+|aug\d+st \d+:\d+:\d+|sep\d+st \d+:\d+:\d+|oct\d+st \d+:\d+:\d+|nov\d+st \d+:\d+:\d+|dec\d+st \d+:\d+:\d+ \
           jan\d+nd \d+:\d+:\d+|feb\d+nd \d+:\d+:\d+|mar\d+nd \d+:\d+:\d+|apr\d+nd \d+:\d+:\d+|may\d+nd \d+:\d+:\d+|jun\d+nd \d+:\d+:\d+|jul\d+nd \d+:\d+:\d+|aug\d+nd \d+:\d+:\d+|sep\d+nd \d+:\d+:\d+|oct\d+nd \d+:\d+:\d+|nov\d+nd \d+:\d+:\d+|dec\d+nd \d+:\d+:\d+ \
           jan\d+rd \d+:\d+:\d+|feb\d+rd \d+:\d+:\d+|mar\d+rd \d+:\d+:\d+|apr\d+rd \d+:\d+:\d+|may\d+rd \d+:\d+:\d+|jun\d+rd \d+:\d+:\d+|jul\d+rd \d+:\d+:\d+|aug\d+rd \d+:\d+:\d+|sep\d+rd \d+:\d+:\d+|oct\d+rd \d+:\d+:\d+|nov\d+rd \d+:\d+:\d+|dec\d+rd \d+:\d+:\d+ \
           jan\d+th \d+:\d+:\d+|feb\d+th \d+:\d+:\d+|mar\d+th \d+:\d+:\d+|apr\d+th \d+:\d+:\d+|may\d+th \d+:\d+:\d+|jun\d+th \d+:\d+:\d+|jul\d+th \d+:\d+:\d+|aug\d+th \d+:\d+:\d+|sep\d+th \d+:\d+:\d+|oct\d+th \d+:\d+:\d+|nov\d+th \d+:\d+:\d+|dec\d+th \d+:\d+:\d+ \
           jan\d+ \d+:\d+:\d+|feb\d+ \d+:\d+:\d+|mar\d+ \d+:\d+:\d+|apr\d+ \d+:\d+:\d+|may\d+ \d+:\d+:\d+|jun\d+ \d+:\d+:\d+|jul\d+ \d+:\d+:\d+|aug\d+ \d+:\d+:\d+|sep\d+ \d+:\d+:\d+|oct\d+ \d+:\d+:\d+|nov\d+ \d+:\d+:\d+|dec\d+ \d+:\d+:\d+"
        for match in re.findall(MonDD_HMS, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            datetime_object = datetime.strptime(match_rep,"%b%d %H:%M:%S")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
        MonDD_HM = r"jan\d+st \d+:\d+|feb\d+st \d+:\d+|mar\d+st \d+:\d+|apr\d+st \d+:\d+|may\d+st \d+:\d+|jun\d+st \d+:\d+|jul\d+st \d+:\d+|aug\d+st \d+:\d+|sep\d+st \d+:\d+|oct\d+st \d+:\d+|nov\d+st \d+:\d+|dec\d+st \d+:\d+ \
           jan\d+nd \d+:\d+|feb\d+nd \d+:\d+|mar\d+nd \d+:\d+|apr\d+nd \d+:\d+|may\d+nd \d+:\d+|jun\d+nd \d+:\d+|jul\d+nd \d+:\d+|aug\d+nd \d+:\d+|sep\d+nd \d+:\d+|oct\d+nd \d+:\d+|nov\d+nd \d+:\d+|dec\d+nd \d+:\d+ \
           jan\d+rd \d+:\d+|feb\d+rd \d+:\d+|mar\d+rd \d+:\d+|apr\d+rd \d+:\d+|may\d+rd \d+:\d+|jun\d+rd \d+:\d+|jul\d+rd \d+:\d+|aug\d+rd \d+:\d+|sep\d+rd \d+:\d+|oct\d+rd \d+:\d+|nov\d+rd \d+:\d+|dec\d+rd \d+:\d+ \
           jan\d+th \d+:\d+|feb\d+th \d+:\d+|mar\d+th \d+:\d+|apr\d+th \d+:\d+|may\d+th \d+:\d+|jun\d+th \d+:\d+|jul\d+th \d+:\d+|aug\d+th \d+:\d+|sep\d+th \d+:\d+|oct\d+th \d+:\d+|nov\d+th \d+:\d+|dec\d+th \d+:\d+ \
           jan\d+ \d+:\d+|feb\d+ \d+:\d+|mar\d+ \d+:\d+|apr\d+ \d+:\d+|may\d+ \d+:\d+|jun\d+ \d+:\d+|jul\d+ \d+:\d+|aug\d+ \d+:\d+|sep\d+ \d+:\d+|oct\d+ \d+:\d+|nov\d+ \d+:\d+|dec\d+ \d+:\d+"
           
        for match in re.findall(MonDD_HM, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            datetime_object = datetime.strptime(match_rep,"%b%d %H:%M")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
        MonDD = r"jan\d+st|feb\d+st|mar\d+st|apr\d+st|may\d+st|jun\d+st|jul\d+st|aug\d+st|sep\d+st|oct\d+st|nov\d+st|dec\d+st \
           jan\d+nd|feb\d+nd|mar\d+nd|apr\d+nd|may\d+nd|jun\d+nd|jul\d+nd|aug\d+nd|sep\d+nd|oct\d+nd|nov\d+nd|dec\d+nd \
           jan\d+rd|feb\d+rd|mar\d+rd|apr\d+rd|may\d+rd|jun\d+rd|jul\d+rd|aug\d+rd|sep\d+rd|oct\d+rd|nov\d+rd|dec\d+rd \
           jan\d+th|feb\d+th|mar\d+th|apr\d+th|may\d+th|jun\d+th|jul\d+th|aug\d+th|sep\d+th|oct\d+th|nov\d+th|dec\d+th \
           jan\d+|feb\d+|mar\d+|apr\d+|may\d+|jun\d+|jul\d+|aug\d+|sep\d+|oct\d+|nov\d+|dec\d+"
        for match in re.findall(MonDD, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            datetime_object = datetime.strptime(match_rep,"%b%d")
            dateNewFormat = datetime_object.strftime("%B%#d")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        # Space between month and date
        Mon_DD_HMS = r"jan \d+st \d+:\d+:\d+|feb \d+st \d+:\d+:\d+|mar \d+st \d+:\d+:\d+|apr \d+st \d+:\d+:\d+|may \d+st \d+:\d+:\d+|jun \d+st \d+:\d+:\d+|jul \d+st \d+:\d+:\d+|aug \d+st \d+:\d+:\d+|sep \d+st \d+:\d+:\d+|oct \d+st \d+:\d+:\d+|nov \d+st \d+:\d+:\d+|dec \d+st \d+:\d+:\d+ \
           jan \d+nd \d+:\d+:\d+|feb \d+nd \d+:\d+:\d+|mar \d+nd \d+:\d+:\d+|apr \d+nd \d+:\d+:\d+|may \d+nd \d+:\d+:\d+|jun \d+nd \d+:\d+:\d+|jul \d+nd \d+:\d+:\d+|aug \d+nd \d+:\d+:\d+|sep \d+nd \d+:\d+:\d+|oct \d+nd \d+:\d+:\d+|nov \d+nd \d+:\d+:\d+|dec \d+nd \d+:\d+:\d+ \
           jan \d+rd \d+:\d+:\d+|feb \d+rd \d+:\d+:\d+|mar \d+rd \d+:\d+:\d+|apr \d+rd \d+:\d+:\d+|may \d+rd \d+:\d+:\d+|jun \d+rd \d+:\d+:\d+|jul \d+rd \d+:\d+:\d+|aug \d+rd \d+:\d+:\d+|sep \d+rd \d+:\d+:\d+|oct \d+rd \d+:\d+:\d+|nov \d+rd \d+:\d+:\d+|dec \d+rd \d+:\d+:\d+ \
           jan \d+th \d+:\d+:\d+|feb \d+th \d+:\d+:\d+|mar \d+th \d+:\d+:\d+|apr \d+th \d+:\d+:\d+|may \d+th \d+:\d+:\d+|jun \d+th \d+:\d+:\d+|jul \d+th \d+:\d+:\d+|aug \d+th \d+:\d+:\d+|sep \d+th \d+:\d+:\d+|oct \d+th \d+:\d+:\d+|nov \d+th \d+:\d+:\d+|dec \d+th \d+:\d+:\d+ \
           jan \d+ \d+:\d+:\d+|feb \d+ \d+:\d+:\d+|mar \d+ \d+:\d+:\d+|apr \d+ \d+:\d+:\d+|may \d+ \d+:\d+:\d+|jun \d+ \d+:\d+:\d+|jul \d+ \d+:\d+:\d+|aug \d+ \d+:\d+:\d+|sep \d+ \d+:\d+:\d+|oct \d+ \d+:\d+:\d+|nov \d+ \d+:\d+:\d+|dec \d+ \d+:\d+:\d+"
        for match in re.findall(Mon_DD_HMS, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            datetime_object = datetime.strptime(match_rep,"%b %d %H:%M:%S")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
        Mon_DD_HM = r"jan \d+st \d+:\d+|feb \d+st \d+:\d+|mar \d+st \d+:\d+|apr \d+st \d+:\d+|may \d+st \d+:\d+|jun \d+st \d+:\d+|jul \d+st \d+:\d+|aug \d+st \d+:\d+|sep \d+st \d+:\d+|oct \d+st \d+:\d+|nov \d+st \d+:\d+|dec \d+st \d+:\d+ \
           jan \d+nd \d+:\d+|feb \d+nd \d+:\d+|mar \d+nd \d+:\d+|apr \d+nd \d+:\d+|may \d+nd \d+:\d+|jun \d+nd \d+:\d+|jul \d+nd \d+:\d+|aug \d+nd \d+:\d+|sep \d+nd \d+:\d+|oct \d+nd \d+:\d+|nov \d+nd \d+:\d+|dec \d+nd \d+:\d+ \
           jan \d+rd \d+:\d+|feb \d+rd \d+:\d+|mar \d+rd \d+:\d+|apr \d+rd \d+:\d+|may \d+rd \d+:\d+|jun \d+rd \d+:\d+|jul \d+rd \d+:\d+|aug \d+rd \d+:\d+|sep \d+rd \d+:\d+|oct \d+rd \d+:\d+|nov \d+rd \d+:\d+|dec \d+rd \d+:\d+ \
           jan \d+th \d+:\d+|feb \d+th \d+:\d+|mar \d+th \d+:\d+|apr \d+th \d+:\d+|may \d+th \d+:\d+|jun \d+th \d+:\d+|jul \d+th \d+:\d+|aug \d+th \d+:\d+|sep \d+th \d+:\d+|oct \d+th \d+:\d+|nov \d+th \d+:\d+|dec \d+th \d+:\d+ \
           jan \d+ \d+:\d+|feb \d+ \d+:\d+|mar \d+ \d+:\d+|apr \d+ \d+:\d+|may \d+ \d+:\d+|jun \d+ \d+:\d+|jul \d+ \d+:\d+|aug \d+ \d+:\d+|sep \d+ \d+:\d+|oct \d+ \d+:\d+|nov \d+ \d+:\d+|dec \d+ \d+:\d+"
        for match in re.findall(Mon_DD_HM, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            datetime_object = datetime.strptime(match_rep,"%b %d %H:%M")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        Mon_DD = r"jan \d+st|feb \d+st|mar \d+st|apr \d+st|may \d+st|jun \d+st|jul \d+st|aug \d+st|sep \d+st|oct \d+st|nov \d+st|dec \d+st \
           jan \d+nd|feb \d+nd|mar \d+nd|apr \d+nd|may \d+nd|jun \d+nd|jul \d+nd|aug \d+nd|sep \d+nd|oct \d+nd|nov \d+nd|dec \d+nd \
           jan \d+rd|feb \d+rd|mar \d+rd|apr \d+rd|may \d+rd|jun \d+rd|jul \d+rd|aug \d+rd|sep \d+rd|oct \d+rd|nov \d+rd|dec \d+rd \
           jan \d+th|feb \d+th|mar \d+th|apr \d+th|may \d+th|jun \d+th|jul \d+th|aug \d+th|sep \d+th|oct \d+th|nov \d+th|dec \d+th \
           jan \d+|feb \d+|mar \d+|apr \d+|may \d+|jun \d+|jul \d+|aug \d+|sep \d+|oct \d+|nov \d+|dec \d+"
        for match in re.findall(Mon_DD, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            datetime_object = datetime.strptime(match_rep,"%b %d")
            dateNewFormat = datetime_object.strftime("%B%#d")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        # Space between month and date
        Month_DD_HMS = r"january \d+st \d+:\d+:\d+|february \d+st \d+:\d+:\d+|march \d+st \d+:\d+:\d+|april \d+st \d+:\d+:\d+|may \d+st \d+:\d+:\d+|june \d+st \d+:\d+:\d+|july \d+st \d+:\d+:\d+|august \d+st \d+:\d+:\d+|september \d+st \d+:\d+:\d+|october \d+st \d+:\d+:\d+|november \d+st \d+:\d+:\d+|december \d+st \d+:\d+:\d+ \
           january \d+nd \d+:\d+:\d+|february \d+nd \d+:\d+:\d+|march \d+nd \d+:\d+:\d+|april \d+nd \d+:\d+:\d+|may \d+nd \d+:\d+:\d+|june \d+nd \d+:\d+:\d+|july \d+nd \d+:\d+:\d+|august \d+nd \d+:\d+:\d+|september \d+nd \d+:\d+:\d+|october \d+nd \d+:\d+:\d+|november \d+nd \d+:\d+:\d+|december \d+nd \d+:\d+:\d+ \
           january \d+rd \d+:\d+:\d+|february \d+rd \d+:\d+:\d+|march \d+rd \d+:\d+:\d+|april \d+rd \d+:\d+:\d+|may \d+rd \d+:\d+:\d+|june \d+rd \d+:\d+:\d+|july \d+rd \d+:\d+:\d+|august \d+rd \d+:\d+:\d+|september \d+rd \d+:\d+:\d+|october \d+rd \d+:\d+:\d+|november \d+rd \d+:\d+:\d+|december \d+rd \d+:\d+:\d+ \
           january \d+th \d+:\d+:\d+|february \d+th \d+:\d+:\d+|march \d+th \d+:\d+:\d+|april \d+th \d+:\d+:\d+|may \d+th \d+:\d+:\d+|june \d+th \d+:\d+:\d+|july \d+th \d+:\d+:\d+|august \d+th \d+:\d+:\d+|september \d+th \d+:\d+:\d+|october \d+th \d+:\d+:\d+|november \d+th \d+:\d+:\d+|december \d+th \d+:\d+:\d+ \
           january \d+ \d+:\d+:\d+|february \d+ \d+:\d+:\d+|march \d+ \d+:\d+:\d+|april \d+ \d+:\d+:\d+|may \d+ \d+:\d+:\d+|june \d+ \d+:\d+:\d+|july \d+ \d+:\d+:\d+|august \d+ \d+:\d+:\d+|september \d+ \d+:\d+:\d+|october \d+ \d+:\d+:\d+|november \d+ \d+:\d+:\d+|december \d+ \d+:\d+:\d+ "
        for match in re.findall(Month_DD_HMS, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            match_rep = re.sub("augu","august",match_rep)
            datetime_object = datetime.strptime(match_rep,"%B %d %H:%M:%S")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        Month_DD_HM = r"january \d+st \d+:\d+|february \d+st \d+:\d+|march \d+st \d+:\d+|april \d+st \d+:\d+|may \d+st \d+:\d+|june \d+st \d+:\d+|july \d+st \d+:\d+|august \d+st \d+:\d+|september \d+st \d+:\d+|october \d+st \d+:\d+|november \d+st \d+:\d+|december \d+st \d+:\d+ \
           january \d+nd \d+:\d+|february \d+nd \d+:\d+|march \d+nd \d+:\d+|april \d+nd \d+:\d+|may \d+nd \d+:\d+|june \d+nd \d+:\d+|july \d+nd \d+:\d+|august \d+nd \d+:\d+|september \d+nd \d+:\d+|october \d+nd \d+:\d+|november \d+nd \d+:\d+|december \d+nd \d+:\d+ \
           january \d+rd \d+:\d+|february \d+rd \d+:\d+|march \d+rd \d+:\d+|april \d+rd \d+:\d+|may \d+rd \d+:\d+|june \d+rd \d+:\d+|july \d+rd \d+:\d+|august \d+rd \d+:\d+|september \d+rd \d+:\d+|october \d+rd \d+:\d+|november \d+rd \d+:\d+|december \d+rd \d+:\d+ \
           january \d+th \d+:\d+|february \d+th \d+:\d+|march \d+th \d+:\d+|april \d+th \d+:\d+|may \d+th \d+:\d+|june \d+th \d+:\d+|july \d+th \d+:\d+|august \d+th \d+:\d+|september \d+th \d+:\d+|october \d+th \d+:\d+|november \d+th \d+:\d+|december \d+th \d+:\d+ \
           january \d+ \d+:\d+|february \d+ \d+:\d+|march \d+ \d+:\d+|april \d+ \d+:\d+|may \d+ \d+:\d+|june \d+ \d+:\d+|july \d+ \d+:\d+|august \d+ \d+:\d+|september \d+ \d+:\d+|october \d+ \d+:\d+|november \d+ \d+:\d+|december \d+ \d+:\d+ "
        for match in re.findall(Month_DD_HM, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            match_rep = re.sub("augu","august",match_rep)
            datetime_object = datetime.strptime(match_rep,"%B %d %H:%M")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
        Month_DD = r"january \d+st|february \d+st|march \d+st|april \d+st|may \d+st|june \d+st|july \d+st|august \d+st|september \d+st|october \d+st|november \d+st|december \d+st \
           january \d+nd|february \d+nd|march \d+nd|april \d+nd|may \d+nd|june \d+nd|july \d+nd|august \d+nd|september \d+nd|october \d+nd|november \d+nd|december \d+nd \
           january \d+rd|february \d+rd|march \d+rd|april \d+rd|may \d+rd|june \d+rd|july \d+rd|august \d+rd|september \d+rd|october \d+rd|november \d+rd|december \d+rd \
           january \d+th|february \d+th|march \d+th|april \d+th|may \d+th|june \d+th|july \d+th|august \d+th|september \d+th|october \d+th|november \d+th|december \d+th \
           january \d+|february \d+|march \d+|april \d+|may \d+|june \d+|july \d+|august \d+|september \d+|october \d+|november \d+|december \d+"
        for match in re.findall(Month_DD, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            match_rep = re.sub("augu","august",match_rep)
            datetime_object = datetime.strptime(match_rep,"%B %d")
            dateNewFormat = datetime_object.strftime("%B%#d")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        # No Space between month and date
        MonthDD_HMS = r"january\d+st \d+:\d+:\d+|february\d+st \d+:\d+:\d+|march\d+st \d+:\d+:\d+|april\d+st \d+:\d+:\d+|may\d+st \d+:\d+:\d+|june\d+st \d+:\d+:\d+|july\d+st \d+:\d+:\d+|august\d+st \d+:\d+:\d+|september\d+st \d+:\d+:\d+|october\d+st \d+:\d+:\d+|november\d+st \d+:\d+:\d+|december\d+st \d+:\d+:\d+ \
           january\d+nd \d+:\d+:\d+|february\d+nd \d+:\d+:\d+|march\d+nd \d+:\d+:\d+|april\d+nd \d+:\d+:\d+|may\d+nd \d+:\d+:\d+|june\d+nd \d+:\d+:\d+|july\d+nd \d+:\d+:\d+|august\d+nd \d+:\d+:\d+|september\d+nd \d+:\d+:\d+|october\d+nd \d+:\d+:\d+|november\d+nd \d+:\d+:\d+|december\d+nd \d+:\d+:\d+ \
           january\d+rd \d+:\d+:\d+|february\d+rd \d+:\d+:\d+|march\d+rd \d+:\d+:\d+|april\d+rd \d+:\d+:\d+|may\d+rd \d+:\d+:\d+|june\d+rd \d+:\d+:\d+|july\d+rd \d+:\d+:\d+|august\d+rd \d+:\d+:\d+|september\d+rd \d+:\d+:\d+|october\d+rd \d+:\d+:\d+|november\d+rd \d+:\d+:\d+|december\d+rd \d+:\d+:\d+ \
           january\d+th \d+:\d+:\d+|february\d+th \d+:\d+:\d+|march\d+th \d+:\d+:\d+|april\d+th \d+:\d+:\d+|may\d+th \d+:\d+:\d+|june\d+th \d+:\d+:\d+|july\d+th \d+:\d+:\d+|august\d+th \d+:\d+:\d+|september\d+th \d+:\d+:\d+|october\d+th \d+:\d+:\d+|november\d+th \d+:\d+:\d+|december\d+th \d+:\d+:\d+ \
           january\d+ \d+:\d+:\d+|february\d+ \d+:\d+:\d+|march\d+ \d+:\d+:\d+|april\d+ \d+:\d+:\d+|may\d+ \d+:\d+:\d+|june\d+ \d+:\d+:\d+|july\d+ \d+:\d+:\d+|august\d+ \d+:\d+:\d+|september\d+ \d+:\d+:\d+|october\d+ \d+:\d+:\d+|november\d+ \d+:\d+:\d+|december\d+ \d+:\d+:\d+ "
        for match in re.findall(MonthDD_HMS, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            match_rep = re.sub("augu","august",match_rep)
            datetime_object = datetime.strptime(match_rep,"%B%d %H:%M:%S")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        MonthDD_HM = r"january\d+st \d+:\d+|february\d+st \d+:\d+|march\d+st \d+:\d+|april\d+st \d+:\d+|may\d+st \d+:\d+|june\d+st \d+:\d+|july\d+st \d+:\d+|august\d+st \d+:\d+|september\d+st \d+:\d+|october\d+st \d+:\d+|november\d+st \d+:\d+|december\d+st \d+:\d+ \
           january\d+nd \d+:\d+|february\d+nd \d+:\d+|march\d+nd \d+:\d+|april\d+nd \d+:\d+|may\d+nd \d+:\d+|june\d+nd \d+:\d+|july\d+nd \d+:\d+|august\d+nd \d+:\d+|september\d+nd \d+:\d+|october\d+nd \d+:\d+|november\d+nd \d+:\d+|december\d+nd \d+:\d+ \
           january\d+rd \d+:\d+|february\d+rd \d+:\d+|march\d+rd \d+:\d+|april\d+rd \d+:\d+|may\d+rd \d+:\d+|june\d+rd \d+:\d+|july\d+rd \d+:\d+|august\d+rd \d+:\d+|september\d+rd \d+:\d+|october\d+rd \d+:\d+|november\d+rd \d+:\d+|december\d+rd \d+:\d+ \
           january\d+th \d+:\d+|february\d+th \d+:\d+|march\d+th \d+:\d+|april\d+th \d+:\d+|may\d+th \d+:\d+|june\d+th \d+:\d+|july\d+th \d+:\d+|august\d+th \d+:\d+|september\d+th \d+:\d+|october\d+th \d+:\d+|november\d+th \d+:\d+|december\d+th \d+:\d+ \
           january\d+ \d+:\d+|februaryruary\d+ \d+:\d+|march\d+ \d+:\d+|april\d+ \d+:\d+|may\d+ \d+:\d+|june\d+ \d+:\d+|july\d+ \d+:\d+|august\d+ \d+:\d+|september\d+ \d+:\d+|october\d+ \d+:\d+|november\d+ \d+:\d+|december\d+ \d+:\d+ "
        for match in re.findall(MonthDD_HM, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            match_rep = re.sub("augu","august",match_rep)
            datetime_object = datetime.strptime(match_rep,"%B%d %H:%M")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        MonthDD = r"january\d+st|february\d+st|march\d+st|april\d+st|may\d+st|june\d+st|july\d+st|august\d+st|september\d+st|october\d+st|november\d+st|december\d+st \
           january\d+nd|february\d+nd|march\d+nd|april\d+nd|may\d+nd|june\d+nd|july\d+nd|august\d+nd|september\d+nd|october\d+nd|november\d+nd|december\d+nd \
           january\d+rd|february\d+rd|march\d+rd|april\d+rd|may\d+rd|june\d+rd|july\d+rd|august\d+rd|september\d+rd|october\d+rd|november\d+rd|december\d+rd \
           january\d+th|february\d+th|march\d+th|april\d+th|may\d+th|june\d+th|july\d+th|august\d+th|september\d+th|october\d+th|november\d+th|december\d+th \
           january\d+|february\d+|march\d+|april\d+|may\d+|june\d+|july\d+|august\d+|september\d+|october\d+|november\d+|december\d+ "
        for match in re.findall(MonthDD, TextwithDates):
            match_rep = re.sub(suffix,'', match)
            match_rep = re.sub("augu","august",match_rep)
            datetime_object = datetime.strptime(match_rep,"%B%d")
            dateNewFormat = datetime_object.strftime("%B%#d")
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
        # Replace 2/29 to february29
        for match in re.findall("2/29", TextwithDates):
            TextwithDates = re.sub(match, "February29", TextwithDates)

        # First convert YYYY to YY
        for match in re.findall(r'/\d{4}', TextwithDates):
            match = re.sub('/','' , match)
            #convert match to new format
            datetime_object = datetime.strptime(match, "%Y")
            dateNewFormat = datetime_object.strftime("%y")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        # convert %m/%d/%y formatted texts to %m/%d
        for match in re.findall(r'\d+/\d+/\d+', TextwithDates):
            #convert match to new format
            datetime_object = datetime.strptime(match, "%m/%d/%y")
            dateNewFormat = datetime_object.strftime("%#m/%#d")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        # convert %m/%d %H:%M:%S formatted texts to %B%d_%H:%M:%S
        for match in re.findall(r'\d+/\d+ \d+:\d+:\d+', TextwithDates):
            datetime_object = datetime.strptime(match, "%m/%d %H:%M:%S")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        # convert %m/%d %H:%M formatted texts to %B%d_%H:%M
        for match in re.findall(r'\d+/\d+ \d+:\d+', TextwithDates):
            datetime_object = datetime.strptime(match, "%m/%d %H:%M")
            dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        # convert %m/%d formatted texts to %B%d
        for match in re.findall(r'\d+/\d+', TextwithDates):
            datetime_object = datetime.strptime(match, "%m/%d")
            dateNewFormat = datetime_object.strftime("%B%#d")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
        return(TextwithDates)
    except Exception:
        return(TextwithDates)
    
# MonthDD = r"January\d+|February\d+|March\d+|April\d+|May\d+|June\d+|July\d+|August\d+|September\d+|October\d+|November\d+|December\d+"    

In [26]:
# # If error then return the original text itself
# def ChangeDateTimeFormatx(TextwithDates):
#     try:
#         # Replace sept with sep            
#         for match in re.findall("sept \d+", TextwithDates):
#             match_rep = re.sub('sept','sep' ,match)
#             TextwithDates = re.sub(match, match_rep, TextwithDates)
            
#         # Convert jan & january independently     
#         suffix = r"st|nd|rd|th"   
        
#         # Extract Month_Year
#         Month_Year = "january \d{4}|february \d{4}|march \d{4}|april \d{4}|may \d{4}|june \d{4}|july \d{4}|august \d{4}|september \d{4}|october \d{4}|november \d{4}|december \d{4}"
#         for match in re.findall(Month_Year, TextwithDates):
#             datetime_object = datetime.strptime(match,"%B %Y")
#             dateNewFormat = datetime_object.strftime("%Y%B")
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
#         # Mon_Year
#         Mon_Year = "jan \d{4}|feb \d{4}|mar \d{4}|apr \d{4}|may \d{4}|jun \d{4}|jul \d{4}|aug \d{4}|sep \d{4}|oct \d{4}|nov \d{4}|dec \d{4}"
#         for match in re.findall(Mon_Year, TextwithDates):
#             datetime_object = datetime.strptime(match,"%b %Y")
#             dateNewFormat = datetime_object.strftime("%Y%B")
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
#         # Mon_Year
#         Mon_Year = "jan \d+|feb \d+|mar \d+|apr \d+|may \d+|jun \d+|jul \d+|aug \d+|sep \d+|oct \d+|nov \d+|dec \d+"
#         for match in re.findall(Mon_Year, TextwithDates):
#             match_rep = re.sub(suffix,'', match)
#             datetime_object = datetime.strptime(match_rep,"%b %Y")
#             dateNewFormat = datetime_object.strftime("%Y%B")
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
#         # No Space between month and date
#         MonDD_HMS = r"jan\d+st \d+:\d+:\d+|feb\d+st \d+:\d+:\d+|mar\d+st \d+:\d+:\d+|apr\d+st \d+:\d+:\d+|may\d+st \d+:\d+:\d+|jun\d+st \d+:\d+:\d+|jul\d+st \d+:\d+:\d+|aug\d+st \d+:\d+:\d+|sep\d+st \d+:\d+:\d+|oct\d+st \d+:\d+:\d+|nov\d+st \d+:\d+:\d+|dec\d+st \d+:\d+:\d+ \
#            jan\d+nd \d+:\d+:\d+|feb\d+nd \d+:\d+:\d+|mar\d+nd \d+:\d+:\d+|apr\d+nd \d+:\d+:\d+|may\d+nd \d+:\d+:\d+|jun\d+nd \d+:\d+:\d+|jul\d+nd \d+:\d+:\d+|aug\d+nd \d+:\d+:\d+|sep\d+nd \d+:\d+:\d+|oct\d+nd \d+:\d+:\d+|nov\d+nd \d+:\d+:\d+|dec\d+nd \d+:\d+:\d+ \
#            jan\d+rd \d+:\d+:\d+|feb\d+rd \d+:\d+:\d+|mar\d+rd \d+:\d+:\d+|apr\d+rd \d+:\d+:\d+|may\d+rd \d+:\d+:\d+|jun\d+rd \d+:\d+:\d+|jul\d+rd \d+:\d+:\d+|aug\d+rd \d+:\d+:\d+|sep\d+rd \d+:\d+:\d+|oct\d+rd \d+:\d+:\d+|nov\d+rd \d+:\d+:\d+|dec\d+rd \d+:\d+:\d+ \
#            jan\d+th \d+:\d+:\d+|feb\d+th \d+:\d+:\d+|mar\d+th \d+:\d+:\d+|apr\d+th \d+:\d+:\d+|may\d+th \d+:\d+:\d+|jun\d+th \d+:\d+:\d+|jul\d+th \d+:\d+:\d+|aug\d+th \d+:\d+:\d+|sep\d+th \d+:\d+:\d+|oct\d+th \d+:\d+:\d+|nov\d+th \d+:\d+:\d+|dec\d+th \d+:\d+:\d+ \
#            jan\d+ \d+:\d+:\d+|feb\d+ \d+:\d+:\d+|mar\d+ \d+:\d+:\d+|apr\d+ \d+:\d+:\d+|may\d+ \d+:\d+:\d+|jun\d+ \d+:\d+:\d+|jul\d+ \d+:\d+:\d+|aug\d+ \d+:\d+:\d+|sep\d+ \d+:\d+:\d+|oct\d+ \d+:\d+:\d+|nov\d+ \d+:\d+:\d+|dec\d+ \d+:\d+:\d+"
#         for match in re.findall(MonDD_HMS, TextwithDates):
#             match_rep = re.sub(suffix,'', match)
#             datetime_object = datetime.strptime(match_rep,"%b%d %H:%M:%S")
#             dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
#         MonDD_HM = r"jan\d+st \d+:\d+|feb\d+st \d+:\d+|mar\d+st \d+:\d+|apr\d+st \d+:\d+|may\d+st \d+:\d+|jun\d+st \d+:\d+|jul\d+st \d+:\d+|aug\d+st \d+:\d+|sep\d+st \d+:\d+|oct\d+st \d+:\d+|nov\d+st \d+:\d+|dec\d+st \d+:\d+ \
#            jan\d+nd \d+:\d+|feb\d+nd \d+:\d+|mar\d+nd \d+:\d+|apr\d+nd \d+:\d+|may\d+nd \d+:\d+|jun\d+nd \d+:\d+|jul\d+nd \d+:\d+|aug\d+nd \d+:\d+|sep\d+nd \d+:\d+|oct\d+nd \d+:\d+|nov\d+nd \d+:\d+|dec\d+nd \d+:\d+ \
#            jan\d+rd \d+:\d+|feb\d+rd \d+:\d+|mar\d+rd \d+:\d+|apr\d+rd \d+:\d+|may\d+rd \d+:\d+|jun\d+rd \d+:\d+|jul\d+rd \d+:\d+|aug\d+rd \d+:\d+|sep\d+rd \d+:\d+|oct\d+rd \d+:\d+|nov\d+rd \d+:\d+|dec\d+rd \d+:\d+ \
#            jan\d+th \d+:\d+|feb\d+th \d+:\d+|mar\d+th \d+:\d+|apr\d+th \d+:\d+|may\d+th \d+:\d+|jun\d+th \d+:\d+|jul\d+th \d+:\d+|aug\d+th \d+:\d+|sep\d+th \d+:\d+|oct\d+th \d+:\d+|nov\d+th \d+:\d+|dec\d+th \d+:\d+ \
#            jan\d+ \d+:\d+|feb\d+ \d+:\d+|mar\d+ \d+:\d+|apr\d+ \d+:\d+|may\d+ \d+:\d+|jun\d+ \d+:\d+|jul\d+ \d+:\d+|aug\d+ \d+:\d+|sep\d+ \d+:\d+|oct\d+ \d+:\d+|nov\d+ \d+:\d+|dec\d+ \d+:\d+"           
#         for match in re.findall(MonDD_HM, TextwithDates):
#             match_rep = re.sub(suffix,'', match)
#             datetime_object = datetime.strptime(match_rep,"%b%d %H:%M")
#             dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
#         MonDD = r"jan\d+st|feb\d+st|mar\d+st|apr\d+st|may\d+st|jun\d+st|jul\d+st|aug\d+st|sep\d+st|oct\d+st|nov\d+st|dec\d+st \
#            jan\d+nd|feb\d+nd|mar\d+nd|apr\d+nd|may\d+nd|jun\d+nd|jul\d+nd|aug\d+nd|sep\d+nd|oct\d+nd|nov\d+nd|dec\d+nd \
#            jan\d+rd|feb\d+rd|mar\d+rd|apr\d+rd|may\d+rd|jun\d+rd|jul\d+rd|aug\d+rd|sep\d+rd|oct\d+rd|nov\d+rd|dec\d+rd \
#            jan\d+th|feb\d+th|mar\d+th|apr\d+th|may\d+th|jun\d+th|jul\d+th|aug\d+th|sep\d+th|oct\d+th|nov\d+th|dec\d+th \
#            jan\d+|feb\d+|mar\d+|apr\d+|may\d+|jun\d+|jul\d+|aug\d+|sep\d+|oct\d+|nov\d+|dec\d+"
#         for match in re.findall(MonDD, TextwithDates):
#             match_rep = re.sub(suffix,'', match)
#             datetime_object = datetime.strptime(match_rep,"%b%d")
#             dateNewFormat = datetime_object.strftime("%B%#d")
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
#         # Space between month and date
#         Mon_DD_HMS = r"jan \d+st \d+:\d+:\d+|feb \d+st \d+:\d+:\d+|mar \d+st \d+:\d+:\d+|apr \d+st \d+:\d+:\d+|may \d+st \d+:\d+:\d+|jun \d+st \d+:\d+:\d+|jul \d+st \d+:\d+:\d+|aug \d+st \d+:\d+:\d+|sep \d+st \d+:\d+:\d+|oct \d+st \d+:\d+:\d+|nov \d+st \d+:\d+:\d+|dec \d+st \d+:\d+:\d+ \
#            jan \d+nd \d+:\d+:\d+|feb \d+nd \d+:\d+:\d+|mar \d+nd \d+:\d+:\d+|apr \d+nd \d+:\d+:\d+|may \d+nd \d+:\d+:\d+|jun \d+nd \d+:\d+:\d+|jul \d+nd \d+:\d+:\d+|aug \d+nd \d+:\d+:\d+|sep \d+nd \d+:\d+:\d+|oct \d+nd \d+:\d+:\d+|nov \d+nd \d+:\d+:\d+|dec \d+nd \d+:\d+:\d+ \
#            jan \d+rd \d+:\d+:\d+|feb \d+rd \d+:\d+:\d+|mar \d+rd \d+:\d+:\d+|apr \d+rd \d+:\d+:\d+|may \d+rd \d+:\d+:\d+|jun \d+rd \d+:\d+:\d+|jul \d+rd \d+:\d+:\d+|aug \d+rd \d+:\d+:\d+|sep \d+rd \d+:\d+:\d+|oct \d+rd \d+:\d+:\d+|nov \d+rd \d+:\d+:\d+|dec \d+rd \d+:\d+:\d+ \
#            jan \d+th \d+:\d+:\d+|feb \d+th \d+:\d+:\d+|mar \d+th \d+:\d+:\d+|apr \d+th \d+:\d+:\d+|may \d+th \d+:\d+:\d+|jun \d+th \d+:\d+:\d+|jul \d+th \d+:\d+:\d+|aug \d+th \d+:\d+:\d+|sep \d+th \d+:\d+:\d+|oct \d+th \d+:\d+:\d+|nov \d+th \d+:\d+:\d+|dec \d+th \d+:\d+:\d+ \
#            jan \d+ \d+:\d+:\d+|feb \d+ \d+:\d+:\d+|mar \d+ \d+:\d+:\d+|apr \d+ \d+:\d+:\d+|may \d+ \d+:\d+:\d+|jun \d+ \d+:\d+:\d+|jul \d+ \d+:\d+:\d+|aug \d+ \d+:\d+:\d+|sep \d+ \d+:\d+:\d+|oct \d+ \d+:\d+:\d+|nov \d+ \d+:\d+:\d+|dec \d+ \d+:\d+:\d+"
#         for match in re.findall(Mon_DD_HMS, TextwithDates):
#             match_rep = re.sub(suffix,'', match)
#             datetime_object = datetime.strptime(match_rep,"%b %d %H:%M:%S")
#             dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
#         Mon_DD_HM = r"jan \d+st \d+:\d+|feb \d+st \d+:\d+|mar \d+st \d+:\d+|apr \d+st \d+:\d+|may \d+st \d+:\d+|jun \d+st \d+:\d+|jul \d+st \d+:\d+|aug \d+st \d+:\d+|sep \d+st \d+:\d+|oct \d+st \d+:\d+|nov \d+st \d+:\d+|dec \d+st \d+:\d+ \
#            jan \d+nd \d+:\d+|feb \d+nd \d+:\d+|mar \d+nd \d+:\d+|apr \d+nd \d+:\d+|may \d+nd \d+:\d+|jun \d+nd \d+:\d+|jul \d+nd \d+:\d+|aug \d+nd \d+:\d+|sep \d+nd \d+:\d+|oct \d+nd \d+:\d+|nov \d+nd \d+:\d+|dec \d+nd \d+:\d+ \
#            jan \d+rd \d+:\d+|feb \d+rd \d+:\d+|mar \d+rd \d+:\d+|apr \d+rd \d+:\d+|may \d+rd \d+:\d+|jun \d+rd \d+:\d+|jul \d+rd \d+:\d+|aug \d+rd \d+:\d+|sep \d+rd \d+:\d+|oct \d+rd \d+:\d+|nov \d+rd \d+:\d+|dec \d+rd \d+:\d+ \
#            jan \d+th \d+:\d+|feb \d+th \d+:\d+|mar \d+th \d+:\d+|apr \d+th \d+:\d+|may \d+th \d+:\d+|jun \d+th \d+:\d+|jul \d+th \d+:\d+|aug \d+th \d+:\d+|sep \d+th \d+:\d+|oct \d+th \d+:\d+|nov \d+th \d+:\d+|dec \d+th \d+:\d+ \
#            jan \d+ \d+:\d+|feb \d+ \d+:\d+|mar \d+ \d+:\d+|apr \d+ \d+:\d+|may \d+ \d+:\d+|jun \d+ \d+:\d+|jul \d+ \d+:\d+|aug \d+ \d+:\d+|sep \d+ \d+:\d+|oct \d+ \d+:\d+|nov \d+ \d+:\d+|dec \d+ \d+:\d+"
#         for match in re.findall(Mon_DD_HM, TextwithDates):
#             match_rep = re.sub(suffix,'', match)
#             datetime_object = datetime.strptime(match_rep,"%b %d %H:%M")
#             dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
#         Mon_DD = r"jan \d+st|feb \d+st|mar \d+st|apr \d+st|may \d+st|jun \d+st|jul \d+st|aug \d+st|sep \d+st|oct \d+st|nov \d+st|dec \d+st \
#            jan \d+nd|feb \d+nd|mar \d+nd|apr \d+nd|may \d+nd|jun \d+nd|jul \d+nd|aug \d+nd|sep \d+nd|oct \d+nd|nov \d+nd|dec \d+nd \
#            jan \d+rd|feb \d+rd|mar \d+rd|apr \d+rd|may \d+rd|jun \d+rd|jul \d+rd|aug \d+rd|sep \d+rd|oct \d+rd|nov \d+rd|dec \d+rd \
#            jan \d+th|feb \d+th|mar \d+th|apr \d+th|may \d+th|jun \d+th|jul \d+th|aug \d+th|sep \d+th|oct \d+th|nov \d+th|dec \d+th \
#            jan \d+|feb \d+|mar \d+|apr \d+|may \d+|jun \d+|jul \d+|aug \d+|sep \d+|oct \d+|nov \d+|dec \d+"
#         for match in re.findall(Mon_DD, TextwithDates):
#             match_rep = re.sub(suffix,'', match)
#             datetime_object = datetime.strptime(match_rep,"%b %d")
#             dateNewFormat = datetime_object.strftime("%B%#d")
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

#         # Space between month and date
#         Month_DD_HMS = r"january \d+st \d+:\d+:\d+|february \d+st \d+:\d+:\d+|march \d+st \d+:\d+:\d+|april \d+st \d+:\d+:\d+|may \d+st \d+:\d+:\d+|june \d+st \d+:\d+:\d+|july \d+st \d+:\d+:\d+|august \d+st \d+:\d+:\d+|september \d+st \d+:\d+:\d+|october \d+st \d+:\d+:\d+|november \d+st \d+:\d+:\d+|december \d+st \d+:\d+:\d+ \
#            january \d+nd \d+:\d+:\d+|february \d+nd \d+:\d+:\d+|march \d+nd \d+:\d+:\d+|april \d+nd \d+:\d+:\d+|may \d+nd \d+:\d+:\d+|june \d+nd \d+:\d+:\d+|july \d+nd \d+:\d+:\d+|august \d+nd \d+:\d+:\d+|september \d+nd \d+:\d+:\d+|october \d+nd \d+:\d+:\d+|november \d+nd \d+:\d+:\d+|december \d+nd \d+:\d+:\d+ \
#            january \d+rd \d+:\d+:\d+|february \d+rd \d+:\d+:\d+|march \d+rd \d+:\d+:\d+|april \d+rd \d+:\d+:\d+|may \d+rd \d+:\d+:\d+|june \d+rd \d+:\d+:\d+|july \d+rd \d+:\d+:\d+|august \d+rd \d+:\d+:\d+|september \d+rd \d+:\d+:\d+|october \d+rd \d+:\d+:\d+|november \d+rd \d+:\d+:\d+|december \d+rd \d+:\d+:\d+ \
#            january \d+th \d+:\d+:\d+|february \d+th \d+:\d+:\d+|march \d+th \d+:\d+:\d+|april \d+th \d+:\d+:\d+|may \d+th \d+:\d+:\d+|june \d+th \d+:\d+:\d+|july \d+th \d+:\d+:\d+|august \d+th \d+:\d+:\d+|september \d+th \d+:\d+:\d+|october \d+th \d+:\d+:\d+|november \d+th \d+:\d+:\d+|december \d+th \d+:\d+:\d+ \
#            january \d+ \d+:\d+:\d+|february \d+ \d+:\d+:\d+|march \d+ \d+:\d+:\d+|april \d+ \d+:\d+:\d+|may \d+ \d+:\d+:\d+|june \d+ \d+:\d+:\d+|july \d+ \d+:\d+:\d+|august \d+ \d+:\d+:\d+|september \d+ \d+:\d+:\d+|october \d+ \d+:\d+:\d+|november \d+ \d+:\d+:\d+|december \d+ \d+:\d+:\d+ "
#         for match in re.findall(Month_DD_HMS, TextwithDates):
#             match_rep = re.sub(suffix,'', match)
#             match_rep = re.sub("augu","august",match_rep)
#             datetime_object = datetime.strptime(match_rep,"%B %d %H:%M:%S")
#             dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
#         Month_DD_HM = r"january \d+st \d+:\d+|february \d+st \d+:\d+|march \d+st \d+:\d+|april \d+st \d+:\d+|may \d+st \d+:\d+|june \d+st \d+:\d+|july \d+st \d+:\d+|august \d+st \d+:\d+|september \d+st \d+:\d+|october \d+st \d+:\d+|november \d+st \d+:\d+|december \d+st \d+:\d+ \
#            january \d+nd \d+:\d+|february \d+nd \d+:\d+|march \d+nd \d+:\d+|april \d+nd \d+:\d+|may \d+nd \d+:\d+|june \d+nd \d+:\d+|july \d+nd \d+:\d+|august \d+nd \d+:\d+|september \d+nd \d+:\d+|october \d+nd \d+:\d+|november \d+nd \d+:\d+|december \d+nd \d+:\d+ \
#            january \d+rd \d+:\d+|february \d+rd \d+:\d+|march \d+rd \d+:\d+|april \d+rd \d+:\d+|may \d+rd \d+:\d+|june \d+rd \d+:\d+|july \d+rd \d+:\d+|august \d+rd \d+:\d+|september \d+rd \d+:\d+|october \d+rd \d+:\d+|november \d+rd \d+:\d+|december \d+rd \d+:\d+ \
#            january \d+th \d+:\d+|february \d+th \d+:\d+|march \d+th \d+:\d+|april \d+th \d+:\d+|may \d+th \d+:\d+|june \d+th \d+:\d+|july \d+th \d+:\d+|august \d+th \d+:\d+|september \d+th \d+:\d+|october \d+th \d+:\d+|november \d+th \d+:\d+|december \d+th \d+:\d+ \
#            january \d+ \d+:\d+|february \d+ \d+:\d+|march \d+ \d+:\d+|april \d+ \d+:\d+|may \d+ \d+:\d+|june \d+ \d+:\d+|july \d+ \d+:\d+|august \d+ \d+:\d+|september \d+ \d+:\d+|october \d+ \d+:\d+|november \d+ \d+:\d+|december \d+ \d+:\d+ "
#         for match in re.findall(Month_DD_HM, TextwithDates):
#             match_rep = re.sub(suffix,'', match)
#             match_rep = re.sub("augu","august",match_rep)
#             datetime_object = datetime.strptime(match_rep,"%B %d %H:%M")
#             dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
#         Month_DD = r"january \d+st|february \d+st|march \d+st|april \d+st|may \d+st|june \d+st|july \d+st|august \d+st|september \d+st|october \d+st|november \d+st|december \d+st \
#            january \d+nd|february \d+nd|march \d+nd|april \d+nd|may \d+nd|june \d+nd|july \d+nd|august \d+nd|september \d+nd|october \d+nd|november \d+nd|december \d+nd \
#            january \d+rd|february \d+rd|march \d+rd|april \d+rd|may \d+rd|june \d+rd|july \d+rd|august \d+rd|september \d+rd|october \d+rd|november \d+rd|december \d+rd \
#            january \d+th|february \d+th|march \d+th|april \d+th|may \d+th|june \d+th|july \d+th|august \d+th|september \d+th|october \d+th|november \d+th|december \d+th \
#            january \d+|february \d+|march \d+|april \d+|may \d+|june \d+|july \d+|august \d+|september \d+|october \d+|november \d+|december \d+"
#         for match in re.findall(Month_DD, TextwithDates):
#             match_rep = re.sub(suffix,'', match)
#             match_rep = re.sub("augu","august",match_rep)
#             datetime_object = datetime.strptime(match_rep,"%B %d")
#             dateNewFormat = datetime_object.strftime("%B%#d")
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
#         # No Space between month and date
#         MonthDD_HMS = r"january\d+st \d+:\d+:\d+|february\d+st \d+:\d+:\d+|march\d+st \d+:\d+:\d+|april\d+st \d+:\d+:\d+|may\d+st \d+:\d+:\d+|june\d+st \d+:\d+:\d+|july\d+st \d+:\d+:\d+|august\d+st \d+:\d+:\d+|september\d+st \d+:\d+:\d+|october\d+st \d+:\d+:\d+|november\d+st \d+:\d+:\d+|december\d+st \d+:\d+:\d+ \
#            january\d+nd \d+:\d+:\d+|february\d+nd \d+:\d+:\d+|march\d+nd \d+:\d+:\d+|april\d+nd \d+:\d+:\d+|may\d+nd \d+:\d+:\d+|june\d+nd \d+:\d+:\d+|july\d+nd \d+:\d+:\d+|august\d+nd \d+:\d+:\d+|september\d+nd \d+:\d+:\d+|october\d+nd \d+:\d+:\d+|november\d+nd \d+:\d+:\d+|december\d+nd \d+:\d+:\d+ \
#            january\d+rd \d+:\d+:\d+|february\d+rd \d+:\d+:\d+|march\d+rd \d+:\d+:\d+|april\d+rd \d+:\d+:\d+|may\d+rd \d+:\d+:\d+|june\d+rd \d+:\d+:\d+|july\d+rd \d+:\d+:\d+|august\d+rd \d+:\d+:\d+|september\d+rd \d+:\d+:\d+|october\d+rd \d+:\d+:\d+|november\d+rd \d+:\d+:\d+|december\d+rd \d+:\d+:\d+ \
#            january\d+th \d+:\d+:\d+|february\d+th \d+:\d+:\d+|march\d+th \d+:\d+:\d+|april\d+th \d+:\d+:\d+|may\d+th \d+:\d+:\d+|june\d+th \d+:\d+:\d+|july\d+th \d+:\d+:\d+|august\d+th \d+:\d+:\d+|september\d+th \d+:\d+:\d+|october\d+th \d+:\d+:\d+|november\d+th \d+:\d+:\d+|december\d+th \d+:\d+:\d+ \
#            january\d+ \d+:\d+:\d+|february\d+ \d+:\d+:\d+|march\d+ \d+:\d+:\d+|april\d+ \d+:\d+:\d+|may\d+ \d+:\d+:\d+|june\d+ \d+:\d+:\d+|july\d+ \d+:\d+:\d+|august\d+ \d+:\d+:\d+|september\d+ \d+:\d+:\d+|october\d+ \d+:\d+:\d+|november\d+ \d+:\d+:\d+|december\d+ \d+:\d+:\d+ "
#         for match in re.findall(MonthDD_HMS, TextwithDates):
#             match_rep = re.sub(suffix,'', match)
#             match_rep = re.sub("augu","august",match_rep)
#             datetime_object = datetime.strptime(match_rep,"%B%d %H:%M:%S")
#             dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
#         MonthDD_HM = r"january\d+st \d+:\d+|february\d+st \d+:\d+|march\d+st \d+:\d+|april\d+st \d+:\d+|may\d+st \d+:\d+|june\d+st \d+:\d+|july\d+st \d+:\d+|august\d+st \d+:\d+|september\d+st \d+:\d+|october\d+st \d+:\d+|november\d+st \d+:\d+|december\d+st \d+:\d+ \
#            january\d+nd \d+:\d+|february\d+nd \d+:\d+|march\d+nd \d+:\d+|april\d+nd \d+:\d+|may\d+nd \d+:\d+|june\d+nd \d+:\d+|july\d+nd \d+:\d+|august\d+nd \d+:\d+|september\d+nd \d+:\d+|october\d+nd \d+:\d+|november\d+nd \d+:\d+|december\d+nd \d+:\d+ \
#            january\d+rd \d+:\d+|february\d+rd \d+:\d+|march\d+rd \d+:\d+|april\d+rd \d+:\d+|may\d+rd \d+:\d+|june\d+rd \d+:\d+|july\d+rd \d+:\d+|august\d+rd \d+:\d+|september\d+rd \d+:\d+|october\d+rd \d+:\d+|november\d+rd \d+:\d+|december\d+rd \d+:\d+ \
#            january\d+th \d+:\d+|february\d+th \d+:\d+|march\d+th \d+:\d+|april\d+th \d+:\d+|may\d+th \d+:\d+|june\d+th \d+:\d+|july\d+th \d+:\d+|august\d+th \d+:\d+|september\d+th \d+:\d+|october\d+th \d+:\d+|november\d+th \d+:\d+|december\d+th \d+:\d+ \
#            january\d+ \d+:\d+|februaryruary\d+ \d+:\d+|march\d+ \d+:\d+|april\d+ \d+:\d+|may\d+ \d+:\d+|june\d+ \d+:\d+|july\d+ \d+:\d+|august\d+ \d+:\d+|september\d+ \d+:\d+|october\d+ \d+:\d+|november\d+ \d+:\d+|december\d+ \d+:\d+ "
#         for match in re.findall(MonthDD_HM, TextwithDates):
#             match_rep = re.sub(suffix,'', match)
#             match_rep = re.sub("augu","august",match_rep)
#             datetime_object = datetime.strptime(match_rep,"%B%d %H:%M")
#             dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
#         MonthDD = r"january\d+st|february\d+st|march\d+st|april\d+st|may\d+st|june\d+st|july\d+st|august\d+st|september\d+st|october\d+st|november\d+st|december\d+st \
#            january\d+nd|february\d+nd|march\d+nd|april\d+nd|may\d+nd|june\d+nd|july\d+nd|august\d+nd|september\d+nd|october\d+nd|november\d+nd|december\d+nd \
#            january\d+rd|february\d+rd|march\d+rd|april\d+rd|may\d+rd|june\d+rd|july\d+rd|august\d+rd|september\d+rd|october\d+rd|november\d+rd|december\d+rd \
#            january\d+th|february\d+th|march\d+th|april\d+th|may\d+th|june\d+th|july\d+th|august\d+th|september\d+th|october\d+th|november\d+th|december\d+th \
#            january\d+|february\d+|march\d+|april\d+|may\d+|june\d+|july\d+|august\d+|september\d+|october\d+|november\d+|december\d+ "
#         for match in re.findall(MonthDD, TextwithDates):
#             match_rep = re.sub(suffix,'', match)
#             match_rep = re.sub("augu","august",match_rep)
#             datetime_object = datetime.strptime(match_rep,"%B%d")
#             dateNewFormat = datetime_object.strftime("%B%#d")
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
#         # Replace 2/29 to february29
#         for match in re.findall("2/29", TextwithDates):
#             TextwithDates = re.sub(match, "February29", TextwithDates)

#         # First convert YYYY to YY
#         for match in re.findall(r'/\d{4}', TextwithDates):
#             match = re.sub('/','' , match)
#             #convert match to new format
#             datetime_object = datetime.strptime(match, "%Y")
#             dateNewFormat = datetime_object.strftime("%y")
#             #substitute the old date with the new
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

#         # convert %m/%d/%y formatted texts to %m/%d
#         for match in re.findall(r'\d+/\d+/\d+', TextwithDates):
#             #convert match to new format
#             datetime_object = datetime.strptime(match, "%m/%d/%y")
#             dateNewFormat = datetime_object.strftime("%#m/%#d")
#             #substitute the old date with the new
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

#         # convert %m/%d %H:%M:%S formatted texts to %B%d_%H:%M:%S
#         for match in re.findall(r'\d+/\d+ \d+:\d+:\d+', TextwithDates):
#             datetime_object = datetime.strptime(match, "%m/%d %H:%M:%S")
#             dateNewFormat = datetime_object.strftime("%B%#d_%H:%M:%S")
#             #substitute the old date with the new
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

#         # convert %m/%d %H:%M formatted texts to %B%d_%H:%M
#         for match in re.findall(r'\d+/\d+ \d+:\d+', TextwithDates):
#             datetime_object = datetime.strptime(match, "%m/%d %H:%M")
#             dateNewFormat = datetime_object.strftime("%B%#d_%H:%M")
#             #substitute the old date with the new
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

#         # convert %m/%d formatted texts to %B%d
#         for match in re.findall(r'\d+/\d+', TextwithDates):
#             datetime_object = datetime.strptime(match, "%m/%d")
#             dateNewFormat = datetime_object.strftime("%B%#d")
#             #substitute the old date with the new
#             TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
            
#         return(TextwithDates)
#     except Exception:
#         return(TextwithDates)

In [ ]:
# #### Trace back the error ####
# import linecache
# import sys

# def PrintException():
#     exc_type, exc_obj, tb = sys.exc_info()
#     f = tb.tb_frame
#     lineno = tb.tb_lineno
#     filename = f.f_code.co_filename
#     linecache.checkcache(filename)
#     line = linecache.getline(filename, lineno, f.f_globals)
#     print('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj)

# import traceback
# textToSearch = "august 2017 3/125 oneal, imogene - 86yom - escherichia coli uti with bacteremia\n\nallegy: no abx\n\npmh: chf, cad, and ckd, pad with renal stent, parkinson's disease, hx ecoli bacteremia and uti august 2017, bladder suspension, cholecystectomy\n\nsh: currently residning at morning pointe alf\n\ncxc:\n4/27 blood e coli pans\n4/27 urine e coli\n\n\nabx:\nceftriaxone 4/27-\nazithro 4/27-5/1\nvanco 4/27x1\n\n5/1 approp, continue"
# try:
#     print(XChangeDateTimeFormat(textToSearch))
# except:
# #     PrintException()
#     print(traceback.print_last())

In [16]:
# Apply ChangeDateTimeFormat on Text Column
ClinicalNotes_NoDateErrors['Text_DateTimeFormatted'] = ClinicalNotes_NoDateErrors.apply(lambda x: ChangeDateTimeFormat(x.Text), axis=1)

# Export data
ClinicalNotes_NoDateErrors.to_excel('Py DataFrames/ClinicalNotes_NoDateErrors.xlsx',index=False)
ClinicalNotes_NoDateErrors.to_pickle("Py DataFrames/ClinicalNotes_NoDateErrors.pkl")

In [18]:
ClinicalNotes_NoDateErrors.head()

,Date,Source,SourceType,Text,Row,TotalWords,TextLength,NumericInText,Text_DateTimeFormatted
9615,2019-05-02 10:16:43.470,Patient Notes,User Entered,bmi>40,9615,1,6,True,bmi>40
9621,2019-05-02 10:49:42.910,Patient Notes,User Entered,bmi>40,9621,1,6,True,bmi>40
9633,2019-05-03 08:08:41.700,Patient Notes,User Entered,bmi>40,9633,1,6,True,bmi>40
9887,2019-05-25 09:17:58.933,Patient Notes,User Entered,bmi >40,9887,2,7,True,bmi >40
23054,2019-08-06 15:20:40.000,Patient Order Documentation,Documentation,rmd 8-12.,23054,2,9,True,rmd 8-12.


In [ ]:
# Errors in Row - 963 (Dates have not been changed)

In [5]:
# Extract only "Documentation"
ClinicalNotes_Documentation = ClinicalNotes_NoDateErrors[ClinicalNotes_NoDateErrors.SourceType=="Documentation"]

# Sort data by Text, Date
ClinicalNotes_Documentation = ClinicalNotes_Documentation.sort_values(by=['Text','Date'], ascending=[True, True])

# Export data
ClinicalNotes_Documentation.to_pickle("Py DataFrames/ClinicalNotes_Documentation.pkl")
ClinicalNotes_Documentation.to_excel('Py DataFrames/ClinicalNotes_Documentation.xlsx',index=False)

In [19]:
# Extract only "User Entered"
ClinicalNotes_UserEntered = ClinicalNotes_NoDateErrors[ClinicalNotes_NoDateErrors.SourceType=="User Entered"]

# Sort data by Text, Date - Data is sorted to remove duplicates
ClinicalNotes_UserEntered = ClinicalNotes_UserEntered.sort_values(by=['Text','Date'], ascending=[True, True])

# Export data
ClinicalNotes_UserEntered.to_pickle("Py DataFrames/ClinicalNotes_UserEntered.pkl")
ClinicalNotes_UserEntered.to_excel('Py DataFrames/ClinicalNotes_UserEntered.xlsx',index=False)

In [154]:
#### Splitting data directly from ClinicalNotes: Exploding and then applying DateTime function
# Extract only "User Entered"
ClinicalNotes_Explode_UserEntered = ClinicalNotes[ClinicalNotes.SourceType=="User Entered"]

# Split observations  
# One efficient way might be to search for the string and extract only those that matches that string  

### Explode the Text column into multiple rows
# We start with creating a new dataframe from the series with Row as the index

#'\r\n\r\n'
ClinicalNotes_Explode_UserEntered = pd.DataFrame(ClinicalNotes_Explode_UserEntered.Text.str.split('\r\n\r\n').tolist(), index=ClinicalNotes_Explode_UserEntered.Row).stack()
# We now want to get rid of the secondary index
ClinicalNotes_Explode_UserEntered = ClinicalNotes_Explode_UserEntered.reset_index([0, 'Row'])
# The final step is to set the column names as we want them
ClinicalNotes_Explode_UserEntered.columns = ['Row', 'Text']

#'\r\n'
ClinicalNotes_Explode_UserEntered = pd.DataFrame(ClinicalNotes_Explode_UserEntered.Text.str.split('\r\n').tolist(), index=ClinicalNotes_Explode_UserEntered.Row).stack()
# We now want to get rid of the secondary index
ClinicalNotes_Explode_UserEntered = ClinicalNotes_Explode_UserEntered.reset_index([0, 'Row'])
# The final step is to set the column names as we want them
ClinicalNotes_Explode_UserEntered.columns = ['Row', 'Text']

#'\n'
ClinicalNotes_Explode_UserEntered = pd.DataFrame(ClinicalNotes_Explode_UserEntered.Text.str.split('\n').tolist(), index=ClinicalNotes_Explode_UserEntered.Row).stack()
# We now want to get rid of the secondary index
ClinicalNotes_Explode_UserEntered = ClinicalNotes_Explode_UserEntered.reset_index([0, 'Row'])
# The final step is to set the column names as we want them
ClinicalNotes_Explode_UserEntered.columns = ['Row', 'Text']

#'\r'
ClinicalNotes_Explode_UserEntered = pd.DataFrame(ClinicalNotes_Explode_UserEntered.Text.str.split('\r').tolist(), index=ClinicalNotes_Explode_UserEntered.Row).stack()
# We now want to get rid of the secondary index
ClinicalNotes_Explode_UserEntered = ClinicalNotes_Explode_UserEntered.reset_index([0, 'Row'])
# The final step is to set the column names as we want them
ClinicalNotes_Explode_UserEntered.columns = ['Row', 'Text']

# Replace Esp char with /
ClinicalNotes_Explode_UserEntered['Text'] = ClinicalNotes_Explode_UserEntered['Text'].str.replace('\\',"/")

# Apply DateTime
ClinicalNotes_Explode_UserEntered['Text_DateTimeFormatted'] = ClinicalNotes_Explode_UserEntered.apply(lambda x: ChangeDateTimeFormat(x.Text), axis=1)

# Extract only ones with Numerics
ClinicalNotes_Explode_UserEntered_Num = ClinicalNotes_Explode_UserEntered[ClinicalNotes_Explode_UserEntered['Text_DateTimeFormatted'].str.contains(("\d"), regex=True)]

# Removes special characters from the beginning of a string
import re
ClinicalNotes_Explode_UserEntered_Num['Text_DateTimeFormatted'] = ClinicalNotes_Explode_UserEntered_Num['Text_DateTimeFormatted'].str.replace(r"^\W+", "")

# Remove Duplicates
UserEntered_Num = ClinicalNotes_Explode_UserEntered_Num[['Text_DateTimeFormatted']].drop_duplicates()

# Export data
UserEntered_Num.to_excel('Py DataFrames/UserEntered_Num.xlsx',index=False)

In [155]:
UserEntered_Num.to_pickle("Py DataFrames/UserEntered_Num.pkl")

In [5]:
WordsList = r"wbc"
UserEntered_Num_WBC = UserEntered_Num[UserEntered_Num['Text_DateTimeFormatted'].str.contains(WordsList)]
UserEntered_Num_WBC.shape

(2054, 1)

In [32]:
UserEntered_Num_WBC.to_excel('Py DataFrames/UserEntered_Num_WBC.xlsx',index=False)

In [6]:
UserEntered_Num_WBC.to_pickle("Py DataFrames/UserEntered_Num_WBC.pkl")

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
new_stop_words = ['-','patient','–','day','pt',]
stop_words.extend(new_stop_words)

In [14]:
results = set()
UserEntered_Num["Text_DateTimeFormatted"].str.lower().str.split().apply(results.update)
print(len(results))

36523


In [30]:
# Most frequent words
Freqs = pd.Series(' '.join(UserEntered_Num["Text_DateTimeFormatted"]).split()).value_counts()[:40000]
Freqs = pd.DataFrame(Freqs)
Freqs['Word'] = Freqs.index
Freqs['TextLength'] = Freqs['Word'].str.len() 
Freqs.sort_values(by=['TextLength'], ascending=[False])
Freqs = Freqs[Freqs.TextLength<=70]
Freqs.to_excel('Py DataFrames/Freqs.xlsx',index=False)

In [28]:
Freqs['wbcRows'] = Freqs['Word'].str.findall(r'wbc')
# ClinicalNotes_WBC = ClinicalNotes_NoDateErrors[(ClinicalNotes_NoDateErrors['wbc'].str.len())!=0]
Freqs_WBC = Freqs[(Freqs['wbcRows'].str.len())!=0]
Freqs_WBC.to_excel('Py DataFrames/Freqs_WBC.xlsx',index=True)

In [21]:
Freqs['TempRows'] = Freqs['Word'].str.findall(r'temp')
# ClinicalNotes_WBC = ClinicalNotes_NoDateErrors[(ClinicalNotes_NoDateErrors['wbc'].str.len())!=0]
Freqs_Temp = Freqs[(Freqs['TempRows'].str.len())!=0]
Freqs_Temp.to_excel('Py DataFrames/Freqs_Temp.xlsx',index=True)

In [26]:
Freqs['TextLength'] = Freqs['Word'].str.len() 
Freqs.sort_values(by=['TextLength'], ascending=[False])
Freqs = Freqs[Freqs.TextLength<=70]

In [139]:
# aferile is in 1220 exploded rows, but there is no value to them - febrile or pyrexial if the oral temperature exceeds 37.5°C (99.5°F) or the rectal temperature exceeds 38°C (100.5°F)

AttributeError: 'Series' object has no attribute 'rest_index'

In [113]:
# # Extract Observations that have specific words in them
# from nltk import sent_tokenize,word_tokenize

# xyz ="re-send sputum cultures.wbc count increased to 14, left infiltrate appears new. wbc maps remain marginal. will start vanc, cefepime, levaquin for hospital acquired pneumonia."

# searched_words = [r'wbc',r'wbcs',r'\.']
# for w in word_tokenize(xyz):
#     if w in searched_words:
#         print(xyz)

# searched_words = [r'wbc',r'wbcs',r'\.']
# def Rows_SearchWords(TextwithDates):
#     for w in word_tokenize(TextwithDates):
#         if w in searched_words:
#             return(TextwithDates)
# xyz ="re-send sputum cultures.wbc count increased to 14, left infiltrate appears new. wbc maps remain marginal. will start vanc, cefepime, levaquin for hospital acquired pneumonia."
# Rows_SearchWords(xyz)

In [121]:
def Rows_SearchWords(TextwithDates):
    for w in word_tokenize(TextwithDates):
        if w in searched_words:
            return(TextwithDates)
xyz ="re-send sputum cultures.wbc count increased to 14, left infiltrate appears new. wbc maps remain marginal. will start vanc, cefepime, levaquin for hospital acquired pneumonia."
Rows_SearchWords(xyz)

In [131]:
searched_words = [r'wbc',r'wbcs',r'tmax',r'tmin',r'temp ',r'temperature',r'febrile',r'fever',r'afebrile',r'bmi',r'cr cl']

UserEntered_Num['Text_DateTimeFormatted_SearchWords'] = UserEntered_Num.apply(lambda x: Rows_SearchWords(x.Text_DateTimeFormatted), axis=1)

In [132]:
UserEntered_Num.head()

,Text_DateTimeFormatted,Text_DateTimeFormatted_SearchWords
8,bmi > 40,bmi > 40
11,cr cl 33.7,None
12,scr >/= 1.5,None
14,scr today 0.8,None
24,weight 44kg female,None


In [20]:
# Analyze ClinicalNotes_UserEntered for duplicate records - For time being ignore dealing with duplicates and remove duplicates after explosion and work with them 

In [57]:
# Split observations  
# One efficient way might be to search for the string and extract only those that matches that string  

### Explode the Text column into multiple rows
# We start with creating a new dataframe from the series with Row as the index

#'\r\n\r\n'
ClinicalNotes_UserEntered_Explode = pd.DataFrame(ClinicalNotes_UserEntered.Text_DateTimeFormatted.str.split('\r\n\r\n').tolist(), index=ClinicalNotes_UserEntered.Row).stack()
# We now want to get rid of the secondary index
ClinicalNotes_UserEntered_Explode = ClinicalNotes_UserEntered_Explode.reset_index([0, 'Row'])
# The final step is to set the column names as we want them
ClinicalNotes_UserEntered_Explode.columns = ['Row', 'Text_DateTimeFormatted']

#'\r\n'
ClinicalNotes_UserEntered_Explode = pd.DataFrame(ClinicalNotes_UserEntered_Explode.Text_DateTimeFormatted.str.split('\r\n').tolist(), index=ClinicalNotes_UserEntered_Explode.Row).stack()
# We now want to get rid of the secondary index
ClinicalNotes_UserEntered_Explode = ClinicalNotes_UserEntered_Explode.reset_index([0, 'Row'])
# The final step is to set the column names as we want them
ClinicalNotes_UserEntered_Explode.columns = ['Row', 'Text_DateTimeFormatted']

#'\n'
ClinicalNotes_UserEntered_Explode = pd.DataFrame(ClinicalNotes_UserEntered_Explode.Text_DateTimeFormatted.str.split('\n').tolist(), index=ClinicalNotes_UserEntered_Explode.Row).stack()
# We now want to get rid of the secondary index
ClinicalNotes_UserEntered_Explode = ClinicalNotes_UserEntered_Explode.reset_index([0, 'Row'])
# The final step is to set the column names as we want them
ClinicalNotes_UserEntered_Explode.columns = ['Row', 'Text_DateTimeFormatted']

#'\r'
ClinicalNotes_UserEntered_Explode = pd.DataFrame(ClinicalNotes_UserEntered_Explode.Text_DateTimeFormatted.str.split('\r').tolist(), index=ClinicalNotes_UserEntered_Explode.Row).stack()
# We now want to get rid of the secondary index
ClinicalNotes_UserEntered_Explode = ClinicalNotes_UserEntered_Explode.reset_index([0, 'Row'])
# The final step is to set the column names as we want them
ClinicalNotes_UserEntered_Explode.columns = ['Row', 'Text_DateTimeFormatted']

In [22]:
ClinicalNotes_UserEntered_Explode.to_pickle("Py DataFrames/ClinicalNotes_UserEntered_Explode.pkl")

In [23]:
# Check rows for Numeric values
ClinicalNotes_UserEntered_Explode_Num = ClinicalNotes_UserEntered_Explode[ClinicalNotes_UserEntered_Explode['Text_DateTimeFormatted'].str.contains((" \d"), regex=True)]
ClinicalNotes_UserEntered_Explode_Num.to_excel('Py DataFrames/ClinicalNotes_UserEntered_Explode_Num.xlsx',index=False)

In [109]:
# Extract sentences within observations that contains a specific mentioned word
from nltk import sent_tokenize,word_tokenize

xyz ="re-send sputum cultures.wbc count increased to 14, left infiltrate appears new. wbc maps remain marginal. will start vanc, cefepime, levaquin for hospital acquired pneumonia."
searched_words = [r'wbc',r'wbcs',r'\.']
for sent in sent_tokenize(xyz):
    for w in word_tokenize(sent):
        if w.lower() in searched_words:
            print(sent)

wbc maps remain marginal.


In [7]:
MonthDD = r"January\d+|February\d+|March\d+|April\d+|May\d+|June\d+|July\d+|August\d+|September\d+|October\d+|November\d+|December\d+"
ClinicalNotes_UserEntered_Explode_Dates = ClinicalNotes_UserEntered_Explode[ClinicalNotes_UserEntered_Explode['Text_DateTimeFormatted'].str.contains(MonthDD)]

In [8]:
ClinicalNotes_UserEntered_Explode_Dates.to_excel('Py DataFrames/ClinicalNotes_UserEntered_Explode_Dates.xlsx',index=False)

In [12]:
# Check rows for Numeric values
ClinicalNotes_UserEntered_Explode_Dates_Num = ClinicalNotes_UserEntered_Explode_Dates[ClinicalNotes_UserEntered_Explode_Dates['Text_DateTimeFormatted'].str.contains((" \d"), regex=True)]
ClinicalNotes_UserEntered_Explode_Dates_Num.to_excel('Py DataFrames/ClinicalNotes_UserEntered_Explode_Dates_Num.xlsx',index=False)

In [ ]:
ClinicalNotes_UserEntered_Explode_Dates.shape

In [ ]:
# Extract rows that contain wbc
ClinicalNotes_UserEntered_Explode_WBC = ClinicalNotes_UserEntered_Explode[ClinicalNotes_UserEntered_Explode['Text_DateTimeFormatted'].str.contains(r'wbc')]

In [ ]:
# 
ClinicalNotes_UserEntered_Explode_WBC = ClinicalNotes_UserEntered_Explode[ClinicalNotes_UserEntered_Explode['Text_DateTimeFormatted'].str.contains(r'wbc')]

In [ ]:
ClinicalNotes_UserEntered_Explode_WBC.head()

In [ ]:
# Check rows for Numeric values
ClinicalNotes_UserEntered_Explode_WBC['NumericInText'] = ClinicalNotes_UserEntered_Explode_WBC['Text_DateTimeFormatted'].str.contains((" \d"), regex=True)

In [ ]:
ClinicalNotes_UserEntered_Explode_WBCNum

In [ ]:
# Extract rows that contain wbc
ClinicalNotes_WBC = ClinicalNotes_UserEntered_Explode.copy()
ClinicalNotes_WBC['wbcRows'] = ClinicalNotes_WBC['Text_DateTimeFormatted'].str.findall(r'wbc')
# ClinicalNotes_WBC = ClinicalNotes_NoDateErrors[(ClinicalNotes_NoDateErrors['wbc'].str.len())!=0]
ClinicalNotes_WBC = ClinicalNotes_WBC[(ClinicalNotes_WBC['wbcRows'].str.len())!=0]

In [ ]:
ClinicalNotes_WBC.shape

In [ ]:
ClinicalNotes_UserEntered_Explode_WBC

In [ ]:
ClinicalNotes_UserEntered_Explode_WBC

In [ ]:
# Extract sentences within observations that contains a specific mentioned word
import nltk
nltk.download('punkt')

from nltk import sent_tokenize,word_tokenize
searched_words = ['wbc','wbcs']
ClinicalNotes_WBC = ClinicalNotes_UserEntered.copy()


ClinicalNotes_WBC['wbc'] = ClinicalNotes_WBC['Text_DateTimeFormatted'].apply(lambda text: [sent for sent in sent_tokenize(text)
                                       if any(True for w in word_tokenize(sent) 
                                               if w.lower() in searched_words)])
# Extract rows that contain wbc
ClinicalNotes_WBC['wbcRows'] = ClinicalNotes_WBC['Text_DateTimeFormatted'].str.findall(r'wbc')

# ClinicalNotes_WBC = ClinicalNotes_NoDateErrors[(ClinicalNotes_NoDateErrors['wbc'].str.len())!=0]
ClinicalNotes_WBC = ClinicalNotes_WBC[(ClinicalNotes_WBC['wbcRows'].str.len())!=0]

# Extract specific columns
ClinicalNotes_WBC = ClinicalNotes_WBC[['Row','Text','Text_DateTimeFormatted','wbc','wbcRows']]
# Shuffle rows
ClinicalNotes_WBC = ClinicalNotes_WBC.sample(frac=1)
#ClinicalNotes_WBC.to_excel('Py DataFrames/ClinicalNotes_WBC.xlsx',index=False)

In [ ]:
ClinicalNotes_WBC.to_excel('Py DataFrames/ClinicalNotes_WBC.xlsx',index=False)

In [ ]:
ClinicalNotes_WBC.shape

In [ ]:
ClinicalNotes_WBC.describe(include='all')

In [ ]:
ClinicalNotes_WBCRows.shape

In [ ]:
sent_tokenize("3/24 and 3/125 oneal, imogene - 86yom - escherichia coli uti with bacteremia\n\nallegy: no abx\n\npmh: chf, cad, and ckd, pad with renal stent, parkinson's disease, hx ecoli bacteremia and uti august 2017, bladder suspension, cholecystectomy\n\nsh. currently residning at morning pointe alf\n\ncxc:\n4/27 blood e coli pans\n4/27 urine e coli\n\n\nabx:\nceftriaxone 4/27-\nazithro 4/27-5/1\nvanco 4/27x1\n\n5/1 approp, continue")

In [ ]:
ClinicalNotes_error = ClinicalNotes[ClinicalNotes.august!="[]"]
ClinicalNotes[ClinicalNotes['chkText']!='[]']

In [ ]:
ClinicalNotes[pd.isnull(ClinicalNotes['chkText'])]

In [ ]:
ClinicalNotes.head(1)

In [ ]:
# After dates modification years if present needs to be replaced with empty space
# Remove duplicate rows of Text

In [ ]:
# Split datasets and once corrected attach back - 
# One efficient way might be to search for the string and extract only those that matches that string  

### Explode the Text column into multiple rows
# We start with creating a new dataframe from the series with EmployeeId as the index
ClinicalNotes_Explode = pd.DataFrame(ClinicalNotes.Text.str.split('\r\n\r\n').tolist(), index=ClinicalNotes.Row).stack()
# We now want to get rid of the secondary index
ClinicalNotes_Explode = ClinicalNotes_Explode.reset_index([0, 'Row'])
# The final step is to set the column names as we want them
ClinicalNotes_Explode.columns = ['Row', 'Text']

### Explode the Text column into multiple rows
# We start with creating a new dataframe from the series with EmployeeId as the index
ClinicalNotes_Explode = pd.DataFrame(ClinicalNotes_Explode.Text.str.split('\r\n').tolist(), index=ClinicalNotes_Explode.Row).stack()
# We now want to get rid of the secondary index
ClinicalNotes_Explode = ClinicalNotes_Explode.reset_index([0, 'Row'])
# The final step is to set the column names as we want them
ClinicalNotes_Explode.columns = ['Row', 'Text']

### Explode the Text column into multiple rows
# We start with creating a new dataframe from the series with EmployeeId as the index
ClinicalNotes_Explode = pd.DataFrame(ClinicalNotes_Explode.Text.str.split('\n').tolist(), index=ClinicalNotes_Explode.Row).stack()
# We now want to get rid of the secondary index
ClinicalNotes_Explode = ClinicalNotes_Explode.reset_index([0, 'Row'])
# The final step is to set the column names as we want them
ClinicalNotes_Explode.columns = ['Row', 'Text']

### Explode the Text column into multiple rows
# We start with creating a new dataframe from the series with EmployeeId as the index
ClinicalNotes_Explode = pd.DataFrame(ClinicalNotes_Explode.Text.str.split('\r').tolist(), index=ClinicalNotes_Explode.Row).stack()
# We now want to get rid of the secondary index
ClinicalNotes_Explode = ClinicalNotes_Explode.reset_index([0, 'Row'])
# The final step is to set the column names as we want them
ClinicalNotes_Explode.columns = ['Row', 'Text']

### Explode the Text column into multiple rows
# We start with creating a new dataframe from the series with EmployeeId as the index
ClinicalNotes_Explode = pd.DataFrame(ClinicalNotes.Text.str.split('[#!?.]\s').tolist(), index=ClinicalNotes.Row).stack()
# We now want to get rid of the secondary index
ClinicalNotes_Explode = ClinicalNotes_Explode.reset_index([0, 'Row'])
# The final step is to set the column names as we want them
ClinicalNotes_Explode.columns = ['Row', 'Text']


# donot split that data based on full stops as ClinicalNote_extract_candidates_numeric function will take care of it and 
# if start or end of line is date we will miss it
# ### Explode the Text column into multiple rows - Temp = 101.2 will be split at decimals so space to be added after the 
# # We start with creating a new dataframe from the series with EmployeeId as the index
# ClinicalNotes_Explode = pd.DataFrame(ClinicalNotes_Explode.Text.str.split('\. ').tolist(), index=ClinicalNotes_Explode.Row).stack()
# # We now want to get rid of the secondary index
# ClinicalNotes_Explode = ClinicalNotes_Explode.reset_index([0, 'Row'])
# # The final step is to set the column names as we want them
# ClinicalNotes_Explode.columns = ['Row', 'Text']

In [ ]:
ClinicalNotes_Explode.to_pickle("Py DataFrames/ClinicalNotes_Explode.pkl")

In [ ]:
ClinicalNotes_Explode

In [11]:
# 4388, 38839
# ClinicalNotes['Text'].iloc[38839]
ClinicalNotes[ClinicalNotes['Row']==4927].Text

4927    cunningham, james  - 72yom - pna, influenza, uti\n\nallergy: no abx\n\npmh: last hosp admitted from 12/07-12/12 for septic shock 2/2 pna, htn, hld, pad, dm2, copd on 3-4lnc at home, osa on cpap (4cmh2o), recurrent, metastatic bladder cancer currently on chemoradiation (last wednesday was lat chemo), h/o small cell lung cancer in 2009 s/p lobectomy + chemoradiation, h/o prostate cancer s/p prostatectomy, h/o melanoma s/p removal\n\nsh:\n\ncxc:\n12/27 sp gpc, gnr\n12/26 flu pos\n12/26 urine gnr\n12/26 blood ngx1\n\n12/26 qtc 416\n\nabx:\nazithro 12/26-12/31\nmero 12/26-\noselta 12/26-\nvanco 12/26-\n\n12/27 mero is likely an overkill, but considering recent iv abx, hospitalization, chemo and that urine growing gnr will wait for sensitivities and deescalate asap
Name: Text, dtype: object

In [ ]:
# Extract only those strings with wbc
wbc_Explode = ClinicalNotes_Explode[ClinicalNotes_Explode["Text"].str.contains("wbc")]

In [ ]:
# Shuffle rows
wbc_Explode = wbc_Explode.sample(frac=1)

In [ ]:
# Remove rows that have 

In [ ]:
wbc_Explode.to_csv('wbc_Explode.csv', index=False)
wbc_Explode.to_pickle("wbc_Explode.pkl")

In [ ]:
wbc_Explode

In [ ]:
Result = wbc_Explode[~(wbc_Explode["Text"].str.contains(("wbc \d"), regex=True))]
Result = Result[~(Result["Text"].str.contains(("wbc: \d"), regex=True))]

In [ ]:
Result

In [ ]:
Result.shape

In [ ]:
# 7268,24917,776,7956,6236,7476,36225,38732,38832,3097,4411,4723,7674,9654,4722,7743,9688,9675,9681,22572,33745,23004,7746,5431,9022,9030,8602,24427,9071,9072,5604,23914,7406,9070,22172,2534,3897,7441,32772,1943,8684,7452,1635,36088,5635
# 5741, 7268, 24917, 776,28281
# 9615,33876, 3099